In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
#重建索引
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [3]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对该Series调用reindex将会根据新索引进行重新排序。
如果某个值不存在则会出现缺失值
对于缺失值，一般我们要进行插填或填值处理。
method选项可以达到此目的，比如使用ffill可以实现向前充值：

In [4]:
obj3 = pd.Series(["bule", "purple", "yellow"], index=[0, 2, 4])
obj3

0      bule
2    purple
4    yellow
dtype: object

In [5]:
obj3.reindex(np.arange(6), method="ffill")

0      bule
1      bule
2    purple
3    purple
4    yellow
5    yellow
dtype: object

借助DataFrame, reindex可以修改（行）索引、列，也可以同时修改。
只传入一个序列时，会重建索引中的行：

In [6]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [7]:
frame2 = frame.reindex(index=["a", "b", "c", "d"])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [8]:
#用columns关键字重建索引：
states = ["Texas", "Utah", "California"]
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


"Ohio"不在states中，所以这一列被删除。
另一种重建索引的方法是传入新的轴标签作为位置参数，然后用axis关键字对指定轴进行重建索引：

In [9]:
frame.reindex(states, axis="columns")

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


frame.reindex(states, axis="columns")
用于重新排列或过滤 DataFrame 的列（或行），以匹配指定的索引或列名顺序。
这个方法可以用于 DataFrame 的列（axis="columns")或行（axis="index"）。

重新排列列或行：根据指定顺序重新排列 DataFrame 中的列或行。
添加缺失的列或行：如果指定的列或行在原始 DataFrame 中不存在，会被添加且值为缺失值（NaN）。
保持 DataFrame 的一致性：当你需要多个 DataFrame 有相同的列或行顺序时，使用 reindex 很有用。

![jupyter](./5.4.png)

In [10]:
#还可以用loc运算符重建索引，这是最常用的方法，只有在新索引DF中已经存在时，才能这么做
#否则 reindex 会给新标签插入缺失值
frame.loc[["a", "d", "c"], ["California", "Texas"]]

,California,Texas
a,2,1
d,8,7
c,5,4


In [ ]:
frame.loc[["a", "d", "c", "e"], ["California", "Texas"]]
这种写法是错误的，因为loc不能直接创建索引，所以有多的索引进来后会报错TypeError

In [12]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [ ]:
frame.loc[["a", "d", "c"], ["California", "Texas"]]
只是创建了一个数据视图，并不会影响原始的DF中的数据，操作返回的是一个新的DF，
如果对某个对象赋值，则会变化，比如：
frame.loc[["a", "d", "c"], ["California", "Texas"]] = some_value
只是想显示特定行列，可以使用 loc 来提取数据，不会改变原始数据

In [14]:
#删除指定轴上的项
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [15]:
new_obj = obj.drop("c")
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [16]:
obj.drop(["d", "c"])

a    0.0
b    1.0
e    4.0
dtype: float64

删除某条轴上的一个或多个项只要有一个索引数组或不包含这些项的列表，就可以使用reindex方法或基于.loc的索引进行删除。
由于需要执行一些数据整理和集合逻辑操作，因此drop方法返回的是一个在指定轴上删除了指定值的新对象。

对于DF，可以删除任意轴上的索引值，先创建一个DF实例

In [17]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [18]:
#用标签序列调用drop会从行标签（axis=0）删除值
data.drop(index=["Colorado", "Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


这里是drop(index=["Colorado", "Ohio"])，传入指定行 index 进行删除
drop 默认从行标签中删除指定的标签（axis=0）行删除。
可以通过设置 axis=1 从列标签中删除指定的标签。
使用 inplace=True 可以直接修改原 DataFrame，而不是创建新的 DataFrame

In [20]:
#通过传入指定列 columns 删除指定列标签
data.drop(columns=["two"])

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [21]:
#可以传入 axis=0 或 axis=1 进行行、列删除
data.drop("two", axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [22]:
data.drop(["two", "four"], axis="columns")

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


In [23]:
#索引、选取和过滤
#Series索引(obj[...])的工作方式类似于Numpy数组的索引，只不过Series的索引值可以不仅仅是整数。
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [24]:
obj["b"]

1.0

In [25]:
obj[1]

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_9744/2469632899.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[1]


1.0

这个警告是 Pandas 将来会改变 Series.__getitem__ 的行为。
当使用整数索引来访问 Series 对象时，当前版本的 Pandas 允许你通过位置（position）访问值，
但在未来的版本中，这种方式将被弃用，并且整数将总是被视为标签（labels）。

In [26]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [27]:
obj[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

In [28]:
obj[[1, 3]]

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_9744/2982346117.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[[1, 3]]


b    1.0
d    3.0
dtype: float64

In [29]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

即，在DF中搜索时，可以不仅仅只搜索index，也可以是后面的数值，上述写法可以用来选取数据，
但有更可靠的方法，利用loc运算符选取索引值：

In [30]:
obj.loc[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

loc方式更可取，因为用[]进行索引时，针对整数的处理有所不同。
如果索引包含整数，常规的基于[]的索引会将整数用作标签，因此索引的选取取决于数据类型。
如下：

In [32]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=["a", "b", "c"])
print(obj1)
print(obj2)

2    1
0    2
1    3
dtype: int64
a    1
b    2
c    3
dtype: int64


In [33]:
obj1[[0, 1, 2]]

0    2
1    3
2    1
dtype: int64

In [34]:
obj2[[0, 1, 2]]

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_9744/2599987575.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj2[[0, 1, 2]]


a    1
b    2
c    3
dtype: int64

In [ ]:
这个警告信息表明，未来版本的 Pandas 中 Series.__getitem__ 方法将不再支持通过整数键来访问位置对应的值。
相反，整数键将始终被视为标签。如果你想根据位置访问 Series 中的值，应该使用 ser.iloc[pos]。

obj2[[0, 1, 2]]是想要通过位置去访问DF，与obj1不同，obj1的索引是整数，obj2的索引是串，
所以jp判断他为按照位置访问并抛出异常。未来将会不支持直接通过整数键访问位置对应值

什么是键什么是值？
在 Pandas 的 Series 对象中，“键”和“值”分别对应于索引和数据：
	1.键（Key）: 在 Series 中，键通常是指索引（Index），它可以是整数、字符串或其他类型的标识符，用来定位和访问 Series 中的数据。
	2.值（Value）: 值是 Series 中与每个键（索引）关联的数据项。它们构成了 Series 中的实际数据内容。
比如：
obj2 = pd.Series([10, 20, 30], index=['a', 'b', 'c'])
	•键: 'a', 'b', 'c' 是索引，它们是键。
	•值: 10, 20, 30 是与这些键关联的值

obj2['a'] 根据键 'a' 获取值 10。

obj2.iloc[0] 根据位置索引 0 获取对应的值 10。

Pandas 的未来版本中，Series 将要求更加明确区分这两者，
因此需要用 iloc 来根据位置访问值，而直接用方括号（[]）时是根据键（索引）来访问。

In [ ]:
obj2.loc[[0, 1]]

In [ ]:
使用loc时如果索引中不含整数，则表达式obj.loc[[0, 1, 2]]会报错
与loc不同，iloc运算符只使用用整数，loc运算符只使用标签，无论索引是否包含整数，都能使用iloc：

In [38]:
obj1.iloc[[0, 1, 2]]

2    1
0    2
1    3
dtype: int64

obj1.iloc[[0, 1, 2]] 是基于行的位置（即行号）来选择和输出的，而不是基于行的索引值。
iloc 是专门用来按位置索引的，无论行的索引值是什么，iloc 都只会根据指定的位置进行提取。
所以其中的0，1，2其实是位置，而不是索引

In [39]:
obj2.iloc[[0, 1, 2]]

a    1
b    2
c    3
dtype: int64

可以用标签(索引index)进行切片，与py不同的是，loc切片是包含末端的，即[a:b]a行到b行包含a、b行

In [40]:
obj2.loc["b":"c"]

b    2
c    3
dtype: int64

切片方法还可以对Series相应部分进行赋值：

In [41]:
obj2.loc["b":"c"] = 5
obj2

a    1
b    5
c    5
dtype: int64

loc 和 iloc 非函数，而是方括号索引，方括号不仅可以切片索引，还可以对DF多个轴进行索引，
使用单个值或序列对DF进行索引，以获取单列或多列：

In [5]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [43]:
data["two"]

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [46]:
data["Ohio"]

KeyError: 'Ohio'

In [ ]:
data["Ohio"]会报错，因为只能对列操作，而没有"Ohio"这一列，所以会报错，
data要输出列或进行列切片，需要用loc或iloc进行操作，
data.iloc[:, 1:3]  # 选择第 1 到第 2 列（不包括第 3 列）
data.loc[:, 'one':'three']  # 选择从 'one' 到 'three' 的所有列
data.iloc[1:3, 0:2]  # 选择第 1 到第 2 行，第 0 到第 1 列
data.loc['row1':'row3', 'col1':'col3']  # 按标签选择指定行和列

In [44]:
data[["three", "one"]]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


方括号即为切片，切多个时要在方括号里再嵌一个方括号，并且这种索引方法有几种特殊用法，
可以通过切片或bool数组选取数据：

In [45]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [50]:
#data[:2]方括号内与py相同，对其进行切片，但只能进行行切片，列切片要用loc或iloc
print(data["three"] > 5)
data[data["three"] > 5]

Ohio        False
Colorado     True
Utah         True
New York     True
Name: three, dtype: bool


,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
data[data["three"] > 5]
是通过bool序列操作，three列大于5的都为true,然后为True的组成一个新的DF打印出来，
保留行数据

In [51]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [10]:
#将DF内值小于5的都改为0
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [11]:
#利用loc和iloc选取DF
#loc标签索引，iloc整数索引(位置索引)
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [65]:
#loc默认行选取，索引选取，列选取要切片
print(data.loc["Colorado"])
print(data.iloc[1])

one      0
two      5
three    6
four     7
Name: Colorado, dtype: int64
one      0
two      5
three    6
four     7
Name: Colorado, dtype: int64


In [12]:
#iloc是位置索引，默认行选取，列选取要切片
data.iloc[1]

one      0
two      5
three    6
four     7
Name: Colorado, dtype: int64

In [13]:
#多行选取
print(data.loc[["Colorado", "New York"]])
data.iloc[[1, 2]]

          one  two  three  four
Colorado    0    5      6     7
New York   12   13     14    15


,one,two,three,four
Colorado,0,5,6,7
Utah,8,9,10,11


In [14]:
#loc同时选取行、列
print(data.loc["Colorado",["two", "three"]])
data.iloc[[1], [1, 2]]

two      5
three    6
Name: Colorado, dtype: int64


,two,three
Colorado,5,6


In [15]:
#[3, 0 ,1]代表了第三列，第一列，第二列，而不是py中的 3 到 0，step为1
data.iloc[[1], [3, 0, 1]]

,four,one,two
Colorado,7,0,5


In [16]:
data.loc[:"Utah", "two"]

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int64

In [ ]:
data.loc[:"Utah", "two"]
逗号前面是行选取，选取从第一行开始直到"Utah"行停止，
逗号后是列选取，即选取"two"列

In [17]:
data.iloc[:, :3]     #选取所有行，前四列下标 0～3 列,是三列不是四列

,one,two,three
Ohio,0,0,0
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [18]:
data.iloc[: , :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


此段代码选取所有行，前三列，并且令第三列值大于5的设置为True，最后DF只输出True的行

loc可以用bool型数组，但iloc不能使用

In [20]:
data.loc[data.three >= 2]      #第三列大于等于 2 的设置为True，DF输出True的行

,one,two,three,four
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


多种方式可以选取和重排pandas中的数据。
对于DF的数据选取方法，索引选项操作有很多

![jupyter](./5.5.png)

整数索引中的陷阱
处理整数索引的pandas对象与py内置数据结构不同，比如列表和元组。
例如下面的代码你不认为会出错：

In [22]:
ser = pd.Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [23]:
ser[-1]

KeyError: -1

按照py的意思看，应该返回最后一个索引的值，
但程序不知道是位置索引还是寻找索引为整数 -1 的值，
最后没有这个索引，所以报了keyError: -1，键错误。
对于非整数索引，则不会报错，不会产生歧义：

In [24]:
ser2 = pd.Series(np.arange(3.), index=["a", "b", "c"])
ser2[-1]

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_838/821879068.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ser2[-1]


2.0

但是还是会提示不要直接通过直接在方括号填写整数去充当位置索引，需要调用iloc，
如果轴索引含有整数最稳妥的方法就是loc和iloc，为了避免错误最好用iloc和loc

In [25]:
ser.iloc[-1]

2.0

In [26]:
#但切片总是基于整数索引
ser[:2]

0    0.0
1    1.0
dtype: float64

In [28]:
#链式索引中的陷阱
#通过标签或整数索引对行或列赋值
#对”one“ 列赋值 1
data.loc[:, "one"] = 1
data

,one,two,three,four
Ohio,1,0,0,0
Colorado,1,5,6,7
Utah,1,9,10,11
New York,1,13,14,15


In [29]:
#对第三行赋值 5
data.iloc[2] = 5
data

,one,two,three,four
Ohio,1,0,0,0
Colorado,1,5,6,7
Utah,5,5,5,5
New York,1,13,14,15


In [31]:
#对 four 列大于5的都赋值为3
data.loc[data["four"] > 5] = 3
data

,one,two,three,four
Ohio,1,0,0,0
Colorado,3,3,3,3
Utah,5,5,5,5
New York,3,3,3,3


In [33]:
#赋值时进行链式选取
data.loc[data.three == 5]["three"] = 6
data

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_838/1404862955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[data.three == 5]["three"] = 6


,one,two,three,four
Ohio,1,0,0,0
Colorado,3,3,3,3
Utah,5,5,5,5
New York,3,3,3,3


In [ ]:
这段代码想要对第“three”列中所有值为 5 的数赋值为 6，但报错ettingWithCopyWarning: 
因为data.loc[data.three == 5]是选取three列中值为5的行，是一个视图，而不是DF，
所以后面对其赋值的操作是对临时视图做赋值，并不是对DF操作，所以会报错，并且data没有被修改

对于此类，必须要重写链路赋值使用单独的loc操作：

In [35]:
data.loc[data.three == 5, "three"] = 6
data

,one,two,three,four
Ohio,1,0,0,0
Colorado,3,3,3,3
Utah,5,5,6,5
New York,3,3,3,3


因此，在赋值时首要原则是避免链式索引。

5.2.4 算数运算和数据对齐
pandas处理不同的索引要简单，
例如：当对象相加时，如果存在不同的索引对，则结果中是所有索引的并集。

In [4]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=["a", "c", "d", "e"])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],
               index=["a", "c", "e", "f", "g"])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [5]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [6]:
#将它们相加：
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [ ]:
对于不重复叠的标签，内部数据对齐会导入缺失值，缺失值会在运算中传播。
对于DF，对齐操作会同时发生在行和列上：

In [7]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list("bcd"),
                   index=["Ohio", "Texas", "Colorado"])
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list("bde"),
                   index=["Utah", "Ohio", "Texas", "Oregon"])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [8]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [9]:
#两个DF相加后会返回一个新的DF，索引为原来两个DF的并集：
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


In [ ]:
在 df1 + df2 操作后，结果的排序是按索引和列标签的 并集 进行排序的。具体来说：
1.行索引：首先是将 df1 和 df2 的行索引取并集，然后按字母顺序排序，因此结果中的行索引是 ['Colorado', 'Ohio', 'Oregon', 'Texas', 'Utah']。
2.列索引：列索引也是按 df1 和 df2 的列标签取并集后，按字母顺序排序。因此结果中的列索引是 ['b', 'c', 'd', 'e']。

在对数据框进行加法操作时，如果某个位置上两个数据框都有对应值，就执行相加操作；
如果某个位置只有一个数据框有值，则将另一个数据框中对应位置的值视为 NaN，相加的结果也是 NaN。
这个排序的逻辑适用于所有类似的对齐操作（例如加法、减法、乘法等）。

当然，如果将没有共同列或行标签的DF对象相加，结果将全为空：
没有重复列或行，直接会去并集并赋为空

In [10]:
df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"B": [3, 4]})
df1

,A
0,1
1,2


In [11]:
df2

,B
0,3
1,4


In [12]:
df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


In [13]:
#带有填充值的算术方法：
#对不同索引的对象进行算数运算时，当一个坐标轴索引不在另一个另一个坐标轴的索引时，填充一个特殊值，
#使用np.nan赋值为NA，不让它为空。
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list("abcd"))
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list("abcde"))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [14]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [17]:
df2.loc[1, "b"] = np.nan    #“b”列第“1”行的那个值设置为NA
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [ ]:
df2.loc[1, "b"] = np.nan 这个操作是基于 索引 来进行的。
也就是说，它会在索引为 1 的那一行，列名为 "b" 的位置上赋值为 NaN。
loc 是基于标签（包括索引和列名）来定位的，而不是位置。

In [18]:
#将它们相加时，没有重叠的位置就会产生NA值：
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


df1 没有 e 列，df2 没有索引为 3 的行，所以都是NA

In [19]:
#使用 df1 的 add 方法，传入 df2 以及一个fill_value的参数，
#用传入值替换运算中的缺失值：
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [ ]:
df1.add(df2, fill_value=0) 允许你在进行两个 DataFrame 的逐元素相加时，
用 0 替换掉 NaN 来进行计算，避免因为 NaN 而得到结果中的 NaN。
也是就是计算df1 + df2 但填补了空缺值。

df1 + df2 只是简单地逐元素相加，不会处理 NaN，结果中如果任意一个元素是 NaN，那么结果也是 NaN。
df1.add(df2, fill_value=0) 提供了额外的功能，可以在相加之前用 fill_value 替换 NaN，从而得到更完整的结果。

In [23]:
#DF 和 Series的除法
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


In [24]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250,0.200000,0.166667,0.142857
2,0.125,0.111111,0.100000,0.090909


这两个语句是等价的，都是1 / df1 中的所有元素，df
df1.rsub(1),代表用 1 - df1 中的每一个元素
df1.radd(1),代表df1 中的每一个元素都加 1
df1.floordiv(df2)代表df1整除df2，
df1.rfloordiv(df2)代表反向整数，df2整除df1，
整除就是df1 // df2 ，例如：15 // 4 值为 3，只取整，不留余

![jupyter](./5.6.png)

In [28]:
#对于Series和 DF重建索引时，也可以指定不同的填充值：
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


In [29]:
#DF 和 Series 之间的运算：需要遵守一定的规则
arr = np.arange(12.).reshape((3, 4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [30]:
arr[0]

array([0., 1., 2., 3.])

In [32]:
#一个二维数组与某一行的差
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [ ]:
arr - arr[0] 这个操作是 NumPy 中的广播操作，它将 arr 的每一行减去第一行 arr[0]。

广播机制: NumPy 的广播机制允许不同形状的数组在算术操作中能够以智能的方式进行扩展。
    这里，arr[0] 的形状 (4,) ，形状 (4,) 表示一个一维数组，包含 4 个元素。
    被广播到整个数组 arr 的形状 (3, 4)，从而实现逐元素的减法操作。
应用场景: 这种操作常用于数据的标准化或归一化处理。
例如，将每一行都减去第一行可以用来移除某些基准值。

In [3]:
#DF和Series也是一样的：
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
series = frame.iloc[0]
print("series:\n",series)
frame

series:
 b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64


,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [4]:
#默认情况下，DF 和 Series 之间的算术运算会将 Series 的索引匹配到DF 的列，然后向下一直广播：
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [ ]:
在 Pandas 中，Series 的索引是用来标识数据的标签，但它不影响 Series 的维度。
无论是否有索引，Series 都是一维的数据结构。

即第一行减去第一行的[0, 1, 2]后变为[0, 0, 0]
开始广播：第二行减去第一行的[0, 1, 2]变为[0, 3, 3]
        第三行减去第一行的[0, 1, 2]变为[6, 6, 6]
        第四行减去第一行的[0, 1, 2]变为[9, 9, 9]

如果某个索引值在DF的列或Series的索引中找不到，则参与的两个对象就会重建索引以形成并集：

In [5]:
series2 = pd.Series(np.arange(3), index=["b", "e", "f"])
series2

b    0
e    1
f    2
dtype: int64

In [6]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [7]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [ ]:
在 Pandas 中，Series 的索引是用来标识数据的标签，但它不影响 Series 的维度。
无论是否有索引，Series 都是一维的数据结构。

DF 和 Series 的算术操作，优先以 DF 为基准，frame 的列在前，
在 frame + series2 的操作中，result 中的列顺序主要是 frame 的列在前，
series2 的列（如果在 frame 中不存在）会被添加到 result 的最后，最后以并集的方式输出。

即：先由frame确定要输出的DF形状，因为frame没有f列，所以一整列直接就是NA值
其余的就是 b 列和 e 列与 frame 相加，[0, Na, 2, Na] + [0, Na, 1, 2] = [0, Na, 3, Na],
分别对应 b、d、e、c,由广播机制其余的各行也是如此。

In [29]:
#如果想在列上广播且匹配行，则必须使用算术运算方法并指定匹配索引，如：
series3 = frame["d"]
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [30]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [31]:
frame.sub(series3, axis="index")

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


In [ ]:
sub()函数是减，就是用frame DF 减去series3，且series3是第“d”列，也就是用其余列减去“d”列的值，
就比如第一列 b，[0, 3, 6, 9] - [1, 4, 7, 10] = [-1, -1, -1, -1]，
由于广播性质，后面的每一列都会与 d 列做差， frame 的 d 列与series3 做差全为 0，e列做差全为 1

传入的axis=“index”就是用于匹配的轴，这里就是用 DF 的行索引 index 去进行列广播

In [2]:
#函数应用和映射
#np的通用函数也可以操作pd对象：
frame = pd.DataFrame(np.random.standard_normal((4, 3)),
                     columns=list("bde"),
                     index=["Utah", "Ohio", "Texas", "Oregon"])
frame

,b,d,e
Utah,-0.592203,-0.135389,0.316163
Ohio,0.026273,0.283388,2.245468
Texas,-0.642608,0.284337,-1.606869
Oregon,-1.174337,1.117145,1.273347


In [3]:
np.abs(frame)

,b,d,e
Utah,0.592203,0.135389,0.316163
Ohio,0.026273,0.283388,2.245468
Texas,0.642608,0.284337,1.606869
Oregon,1.174337,1.117145,1.273347


In [5]:
#另一种常见的操作是将函数应用到各列各行所形成的一维数组上。DF 的 apply可以实现此功能
def f1(x):
    return x.max() - x.min()

frame.apply(f1）    #列操作，最后返回的是一个Series，frame的列作为索引

b    1.200610
d    1.252534
e    3.852337
dtype: float64

In [ ]:
1.	函数 f1:
    这是一个简单的函数，接收一个 pandas.Series（即 DataFrame 的一列或一行）作为输入，并返回该列或行的最大值和最小值之间的差值。
2.	frame.apply(f1):
    apply 函数会将 f1 函数应用到 DataFrame 的每一列或每一行。
    默认情况下，apply 会将函数应用到 DataFrame 的每一列（axis=0），如果你想对每一行应用函数，可以使用 frame.apply(f1, axis=1)。

In [7]:
frame.apply(f1, axis=1)    #行操作，每一行的最大值减最小值

Utah      0.908366
Ohio      2.219195
Texas     1.891206
Oregon    2.447684
dtype: float64

In [9]:
#对行操作还可以这样操作：
frame.apply(f1, axis="columns")    #

Utah      0.908366
Ohio      2.219195
Texas     1.891206
Oregon    2.447684
dtype: float64

axis="columns"：
这告诉 apply 函数，你希望函数 f1 应用于 DataFrame 的每一行，而不是每一列。
对于每一行，f1 将接收到一个 pandas.Series 对象，其中包含该行的所有数据。

当 axis=1 或者 axis="columns" 时，apply 函数会沿着列的方向移动，这意味着它会取出每一行的数据，然后将函数应用于这整行。

它是“横向”操作，因此需要处理每一行的数据。虽然参数名称是 columns，但它是针对行的操作，因为你在列之间“移动”。

pandas的 axis 操作比平时的操作容易混淆，但这就是pandas的约定

In [11]:
#许多最为常见的数组统计功能（sum、mean）都是DF的方法，因此无需使用apply方法。
#且传递到apply的函数不一定返回单个标量值，还可以反回由多个值组成的Series：
def f2(x):
    return pd.Series([x.min(), x.max()], index=["min", "max"])

frame.apply(f2)

,b,d,e
min,-1.174337,-0.135389,-1.606869
max,0.026273,1.117145,2.245468


In [ ]:
函数 f2
输入: f2 接收一个 pandas.Series 对象，即 DataFrame 的一列（或一行，取决于 apply 的 axis 参数）。
输出: f2 返回一个新的 pandas.Series，其中包含两个元素：该列或行的最小值 (min) 和最大值 (max)。索引被设置为 "min" 和 "max"。

frame.apply(f2)
当使用 apply(f2) 时，f2 函数会被应用到 DataFrame 的每一列，因为 axis=0 是 apply 函数的默认值。
这意味着 f2 会对 DataFrame 的每一列执行最小值和最大值的计算。

In [12]:
#还可以使用元素级的py函数，如frame中的各个浮点值的格式化字符串，使用applymap：
def my_format(x):
    return f"{x:.2f}"

frame.applymap(my_format)

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_1226/904657603.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  frame.applymap(my_format)


,b,d,e
Utah,-0.59,-0.14,0.32
Ohio,0.03,0.28,2.25
Texas,-0.64,0.28,-1.61
Oregon,-1.17,1.12,1.27


In [ ]:
applymap 是 pandas 提供的方法，用于将一个函数应用到 DataFrame 中的每一个元素。
与 apply 和 applymap 不同的是，applymap 作用于 DataFrame 的每一个单独元素，而不是整列或整行。

输入: my_format 接收一个单独的值 x，通常是 DataFrame 中的每一个元素。
输出: 该函数返回一个格式化后的字符串，保留两位小数（"{x:.2f}" 表示将数字格式化为小数点后两位的字符串）。

"{x:.2f}"是接受之前从 DF 中的每一个值，格式化为保留小数点后两位并输出。

In [13]:
#之所以叫applymap，是因为Series有一个用于元素级函数的 map 方法：
frame["e"].map(my_format)

Utah       0.32
Ohio       2.25
Texas     -1.61
Oregon     1.27
Name: e, dtype: object

In [ ]:
map 方法 用于将函数或映射规则应用到 Series 的每一个元素上。

map 方法用于对 Series 对象的每一个元素应用一个函数或替换规则。
具体来说，当你使用 frame["e"].map(my_format) 时，
map 方法会对 frame["e"] 这一列的每一个元素应用 my_format 函数，
并返回一个新的 Series，其中的元素已经按照 my_format 函数进行了处理。

map 会将 my_format 函数应用于 frame["e"] 列的每一个元素，从而得到一个保留两位小数的字符串格式的结果。

map 的作用：
输入: map 方法接收一个函数或一个映射规则（如字典、Series 等）。
输出: 返回一个新的 Series，其中每个元素都已经通过提供的函数或规则进行了转换。

比如用字典做映射规则：
gender_map = {"M": "Male", "F": "Female"}
df["gender"] = df["gender"].map(gender_map)

这会将 M 替换为 "Male"，将 F 替换为 "Female"。

In [14]:
#排序和排名：
#根据特定条件对数据集排序也是一种重要的内置运算。
#要对行或列按字典排序，可以使用sort_index方法，它将返回好一个排好序的新对象
obj = pd.Series(np.arange(4), index=["d", "a", "b", "c"])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [15]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [2]:
#对于 DF，可以对任意一个轴上的索引进行排序：
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index=["three", "one"],
                     columns=["d", "a", "b", "c"])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [3]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [4]:
frame.sort_index(axis="columns")    #等同于axis=1，按列排序

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [5]:
frame.sort_index(axis=0)   #按行排序，默认即为

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [6]:
#数据默认为升序，但也可以降序：
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [7]:
frame.sort_index(axis=0, ascending=False)

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [8]:
#按值对Series排序：
obj = pd.Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

In [9]:
frame.sort_values(by="d", ascending=False)

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [ ]:
pandas 的 DataFrame.sort_values() 方法需要你指定一个 by 参数，这个参数用于确定按照哪一列或哪些列的值进行排序。
by 参数是必须的，没有指定这个参数，就会触发 TypeError。
但Series可以直接用sort_values（）

by 参数：必须指定，告诉 sort_values 方法你希望按哪一列或哪些列排序。
ascending=False：可选参数，用于指定降序排序。

In [10]:
#一般都 DF 按照索引排序的话都是用sort_index()来排序，而不是axis=1通过by去按行排序
frame.sort_values(axis=1, by=["three", "one"], ascending=False)

,c,b,a,d
three,3,2,1,0
one,7,6,5,4


In [11]:
#在排序时，任何缺失值默认都会放到 Series 的末尾：
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

In [12]:
#使用na_position选项可以将缺失值排在最前面：
obj.sort_values(na_position="first")

1    NaN
3    NaN
4   -3.0
5    2.0
0    4.0
2    7.0
dtype: float64

In [13]:
#对 DF 排序时，可以使用一列或多列中的数据作为排序键，将一个或多个列命传递给 sort_value 即可：
frame = pd.DataFrame({"b": [4, 7, -3, 2], "a": [0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [14]:
#根据 b 列的值排序
frame.sort_values("b")

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [15]:
#多列排序
frame.sort_values(["a", "b"])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


In [ ]:
frame.sort_values(["a", "b"]) 
代码会先按 a 列进行排序，如果 a 列中有相同的值，再按 b 列进行排序。
所以，排序的优先级是先按 a 列，然后按 b 列。

1.首先按 a 列排序：
    a 列中的值是 [0, 1, 0, 1]，排序后，a = 0 的行会排在前面，a = 1 的行排在后面。
2.在 a 列相同的情况下，按 b 列排序：
	对于 a = 0 的行，b 列的值分别是 4 和 -3，所以按 b 列排序后，顺序会是 -3 然后 4。
	对于 a = 1 的行，b 列的值分别是 7 和 2，所以按 b 列排序后，顺序会是 2 然后 7。

所以是：先按照 a 列排序，a 列相同的情况下再按照 b 列排序。

In [18]:
#rank通过“各个组分配平均排名”破坏平级关系
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])
print(obj)
obj.rank()

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64


0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

In [21]:
df = pd.Series([7, -5, 7, 4, 2, 0, 4], name="Values")
df

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
Name: Values, dtype: int64

In [22]:
#或者 DF 写法：字典
df = pd.DataFrame({"Values": [7, -5, 7, 4, 2, 0, 4]})
df

,Values
0,7
1,-5
2,7
3,4
4,2
5,0
6,4


In [24]:
#默认排名：
df["rank"] = df["Values"].rank()
df

,Values,rank
0,7,6.5
1,-5,1.0
2,7,6.5
3,4,4.5
4,2,3.0
5,0,2.0
6,4,4.5


In [ ]:
rank 函数就是直接打印出每个索引的排名位置，
比如索引 0 的值为 7， 因为有两个 7 ，所以排名时排在了 6.5 

In [25]:
#根据原数据中出现的顺序给出排名：
obj.rank(method="first")

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

原数据索引 0 和索引 2 的值是相同的，因为现在用了方法，
所以索引 0 位置的值 7 先出现的，就排在了第 6 位， 索引 2 位置就排在了第 7 位
标签 0 在标签 2 的前面，所以 0 排名 6， 2 排名 7 

In [26]:
#降序排名：
obj.rank(ascending=False)

0    1.5
1    7.0
2    1.5
3    3.5
4    5.0
5    6.0
6    3.5
dtype: float64

In [27]:
# DF 可以在行或列上计算排名：
frame = pd.DataFrame({"b": [4.3, 7, -3, 2], "a": [0, 1, 0, 1],
                      "c": [-2, 5, 8, -2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [28]:
frame.rank(axis="columns")  #在行上计算排名

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


可以打破平级排名的几种方法：

![jupyter](./5.7.png)

In [30]:
#带有重复标签的轴索引   pandas很多函数都要求索引值唯一，如 reindex，但并不是强制的
obj = pd.Series(np.arange(5), index=["a", "a", "b", "b", "c"])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

In [31]:
#索引的 is_unique 属性可以告诉我们索引值不是唯一的：
obj.index.is_unique

False

In [32]:
#对于带有重复值的索引，数据选取操作会不同，
#如果标签对应多个项，则返回 Series，如果对应单个项则返回标量值
obj["a"]

a    0
a    1
dtype: int64

In [33]:
obj["c"]

4

In [37]:
#这样会使代码复杂，因为索引输出类型是会根据标签是不是重复的而发生变化。
#对 DF 的行或列索引也是如此：
df = pd.DataFrame(np.random.standard_normal((5, 3)),
                  index=["a", "a", "b", "b", "c"],
                  #columns=range(3)
                  )
df

,0,1,2
a,0.366024,-1.009947,-0.741921
a,0.983182,0.618933,-0.394976
b,1.153341,0.139761,0.451032
b,0.281114,1.249752,0.350718
c,0.127918,1.330364,-0.969428


In [39]:
df.loc["b"]

,0,1,2
b,1.153341,0.139761,0.451032
b,0.281114,1.249752,0.350718


In [40]:
df.loc["c"]

0    0.127918
1    1.330364
2   -0.969428
Name: c, dtype: float64

#5.4 描述性统计的汇总和计算
pands对象有常用的数学和统计方法，用于从Series中提取单个值，比如sum和mean，
或者从行或列中提取Series，
并且与 np 都有补充缺失值的功能。

In [41]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                   index=["a", "b", "c", "d"],
                   columns=["one", "two"])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [51]:
# DF 的 sum 方法：  
df.sum()    # DF 默认为行操作，列展现，axis=0   跨行操作

one    9.25
two   -5.80
dtype: float64

In [50]:
df.sum(axis=1)   #列操作，行展现

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [49]:
df.sum(axis="columns")  #等同于axis=1 各行跨列运算

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [54]:
#如果某行或某列全为Na值，则和为0，如果既有Na又有非Na值，则会自动跳过Na，
#如果不想跳过Na值，通过skipna选项设置。
df.sum(axis="index")

one    9.25
two   -5.80
dtype: float64

In [55]:
df.sum(axis="index", skipna=False)

one   NaN
two   NaN
dtype: float64

In [ ]:
axis=0 或 axis="index"：沿着行（即按行进行操作）。例如，df.sum(axis=0) 会对每一列的数值进行求和。
axis=1 或 axis="columns"：沿着列（即按列进行操作）。例如，df.sum(axis=1) 会对每一行的数值进行求和。

axis="index" 和 axis=0 是等价的，axis="columns" 和 axis=1 也是等价的。

In [58]:
df.sum(axis="columns")

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

In [59]:
df.sum(axis="columns", skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

In [60]:
#但有些链接方法，比如 mean，至少有一个非 Na 的值：
df.mean(axis="columns")

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

因为 a 列只有一个 1.4 和 Na值，所以最后平均值为 1.40 ，c 列两个Na则全为空

简约py代码的一些常用选项：
![jupyter](./5.8.png)

In [61]:
#有些方法返回的是间接统计，如 idxmin 和 idxmax，达到最小值或最大值的索引：
df.idxmax()    #默认axis=0， 留行不留列，找出每行最大的值的那一列所在

one    b
two    d
dtype: object

In [63]:
#累计型
df.cumsum()   

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [ ]:
默认axis=0， 跨行操作，行不变，列从上向下累加，比如 one 列为1.40， 向下加 b 的值，
b 的值为7.10， 进行累加后为8.50，后面依次累加。

In [67]:
#还有一些方法既不是约减也不是累计型。descibe 就属于此类，可以一次生成多个汇总统计：
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


In [68]:
#对于非数值型数据，describe会产生另外一种汇总统计：
obj = pd.Series(["a", "a", "b", "c"] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

In [ ]:
["a", "a", "b", "c"] * 4 实际输出为16位 ["a", "a", "b", "c", "a", "a", "b", "c", "a", "a", "b", "c", "a", "a", "b", "c"]。
describe() 方法用于生成描述性统计信息。对于非数值型（如字符串）的 Series，它返回有关这些数据的统计信息，包括以下内容：
count：非空值的数量。
unique：唯一值的数量。
top：出现次数最多的值。
freq：出现次数最多的值的频率。

所有描述性统计和汇总统计相关方法：

![jupyter](./5.9.png)

In [ ]:
#相关系数与协方差


In [66]:
def decorator1(func):
    def wrapper(*args, **kwargs):
        print("Decorator 1 before function call")
        result = func(*args, **kwargs)
        print("Decorator 1 after function call")
        return result
    return wrapper

def decorator2(func):
    def wrapper(*args, **kwargs):
        print("Decorator 2 before function call")
        result = func(*args, **kwargs)
        print("Decorator 2 after function call")
        return result
    return wrapper

@decorator1
@decorator2
def my_function():
    print("Inside the function")

my_function()

Decorator 1 before function call
Decorator 2 before function call
Inside the function
Decorator 2 after function call
Decorator 1 after function call


In [ ]:
# Copyright (c) <2015-Present> Tzutalin
# Copyright (C) 2013  MIT, Computer Science and Artificial Intelligence Laboratory. Bryan Russell, Antonio Torralba,
# William T. Freeman. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and
# associated documentation files (the "Software"), to deal in the Software without restriction, including without
# limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the
# Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included in all copies or substantial portions of
# the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT
# NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT
# SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF
# CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.
# !/usr/bin/env python
# -*- coding: utf-8 -*-
# pyrcc5 -o libs/resources.py resources.qrc
import argparse
import ast
import codecs
import json
import os.path
import platform
import subprocess
import sys
import xlrd
from functools import partial

from PyQt5.QtCore import QSize, Qt, QPoint, QByteArray, QTimer, QFileInfo, QPointF, QProcess
from PyQt5.QtGui import QImage, QCursor, QPixmap, QImageReader
from PyQt5.QtWidgets import QMainWindow, QListWidget, QVBoxLayout, QToolButton, QHBoxLayout, QDockWidget, QWidget, \
    QSlider, QGraphicsOpacityEffect, QMessageBox, QListView, QScrollArea, QWidgetAction, QApplication, QLabel, QGridLayout, \
    QFileDialog, QListWidgetItem, QComboBox, QDialog, QAbstractItemView, QSizePolicy

__dir__ = os.path.dirname(os.path.abspath(__file__))

sys.path.append(__dir__)
sys.path.append(os.path.abspath(os.path.join(__dir__, '../..')))
sys.path.append(os.path.abspath(os.path.join(__dir__, '../PaddleOCR')))
sys.path.append("..")

from paddleocr import PaddleOCR, PPStructure
from libs.constants import *
from libs.utils import *
from libs.labelColor import label_colormap
from libs.settings import Settings
from libs.shape import Shape, DEFAULT_LINE_COLOR, DEFAULT_FILL_COLOR, DEFAULT_LOCK_COLOR
from libs.stringBundle import StringBundle
from libs.canvas import Canvas
from libs.zoomWidget import ZoomWidget
from libs.autoDialog import AutoDialog
from libs.labelDialog import LabelDialog
from libs.colorDialog import ColorDialog
from libs.ustr import ustr
from libs.hashableQListWidgetItem import HashableQListWidgetItem
from libs.editinlist import EditInList
from libs.unique_label_qlist_widget import UniqueLabelQListWidget
from libs.keyDialog import KeyDialog

__appname__ = 'PPOCRLabel'

LABEL_COLORMAP = label_colormap()


class MainWindow(QMainWindow):
    FIT_WINDOW, FIT_WIDTH, MANUAL_ZOOM = list(range(3))

    def __init__(self,
                 lang="ch",
                 gpu=False,
                 kie_mode=False,
                 default_filename=None,
                 default_predefined_class_file=None,
                 default_save_dir=None):
        super(MainWindow, self).__init__()
        self.setWindowTitle(__appname__)
        self.setWindowState(Qt.WindowMaximized)  # set window max
        self.activateWindow()  # PPOCRLabel goes to the front when activate

        # Load setting in the main thread
        self.settings = Settings()
        self.settings.load()
        settings = self.settings
        self.lang = lang

        # Load string bundle for i18n
        if lang not in ['ch', 'en']:
            lang = 'en'
        self.stringBundle = StringBundle.getBundle(localeStr='zh-CN' if lang == 'ch' else 'en')  # 'en'
        getStr = lambda strId: self.stringBundle.getString(strId)

        # KIE setting
        self.kie_mode = kie_mode
        self.key_previous_text = ""
        self.existed_key_cls_set = set()
        self.key_dialog_tip = getStr('keyDialogTip')

        self.defaultSaveDir = default_save_dir
        self.ocr = PaddleOCR(use_pdserving=False,
                             use_angle_cls=True,
                             det=True,
                             cls=True,
                             use_gpu=gpu,
                             lang=lang,
                             show_log=False)
        self.table_ocr = PPStructure(use_pdserving=False,
                                     use_gpu=gpu,
                                     lang=lang,
                                     layout=False,
                                     show_log=False)

        if os.path.exists('./data/paddle.png'):
            result = self.ocr.ocr('./data/paddle.png', cls=True, det=True)
            result = self.table_ocr('./data/paddle.png', return_ocr_result_in_table=True)

        # For loading all image under a directory
        self.mImgList = []
        self.mImgList5 = []
        self.dirname = None
        self.labelHist = []
        self.lastOpenDir = None
        self.result_dic = []
        self.result_dic_locked = []
        self.changeFileFolder = False
        self.haveAutoReced = False
        self.labelFile = None
        self.currIndex = 0

        # Whether we need to save or not.
        self.dirty = False

        self._noSelectionSlot = False
        self._beginner = True
        self.screencastViewer = self.getAvailableScreencastViewer()
        self.screencast = "https://github.com/PaddlePaddle/PaddleOCR"

        # Load predefined classes to the list
        self.loadPredefinedClasses(default_predefined_class_file)

        # Main widgets and related state.
        self.labelDialog = LabelDialog(parent=self, listItem=self.labelHist)
        self.autoDialog = AutoDialog(parent=self)

        self.itemsToShapes = {}
        self.shapesToItems = {}
        self.itemsToShapesbox = {}
        self.shapesToItemsbox = {}
        self.prevLabelText = getStr('tempLabel')
        self.noLabelText = getStr('nullLabel')
        self.model = 'paddle'
        self.PPreader = None
        self.autoSaveNum = 5

        #  ================== File List  ==================

        filelistLayout = QVBoxLayout()
        filelistLayout.setContentsMargins(0, 0, 0, 0)

        self.fileListWidget = QListWidget()
        self.fileListWidget.itemClicked.connect(self.fileitemDoubleClicked)
        self.fileListWidget.setIconSize(QSize(25, 25))
        filelistLayout.addWidget(self.fileListWidget)

        fileListContainer = QWidget()
        fileListContainer.setLayout(filelistLayout)
        self.fileListName = getStr('fileList')
        self.fileDock = QDockWidget(self.fileListName, self)
        self.fileDock.setObjectName(getStr('files'))
        self.fileDock.setWidget(fileListContainer)
        self.addDockWidget(Qt.LeftDockWidgetArea, self.fileDock)

        #  ================== Key List  ==================
        if self.kie_mode:
            self.keyList = UniqueLabelQListWidget()

            # set key list height
            key_list_height = int(QApplication.desktop().height() // 4)
            if key_list_height < 50:
                key_list_height = 50
            self.keyList.setMaximumHeight(key_list_height)

            self.keyListDockName = getStr('keyListTitle')
            self.keyListDock = QDockWidget(self.keyListDockName, self)
            self.keyListDock.setWidget(self.keyList)
            self.keyListDock.setFeatures(QDockWidget.NoDockWidgetFeatures)
            filelistLayout.addWidget(self.keyListDock)

        self.AutoRecognition = QToolButton()
        self.AutoRecognition.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)
        self.AutoRecognition.setIcon(newIcon('Auto'))
        autoRecLayout = QHBoxLayout()
        autoRecLayout.setContentsMargins(0, 0, 0, 0)
        autoRecLayout.addWidget(self.AutoRecognition)
        autoRecContainer = QWidget()
        autoRecContainer.setLayout(autoRecLayout)
        filelistLayout.addWidget(autoRecContainer)

        #  ================== Right Area  ==================
        listLayout = QVBoxLayout()
        listLayout.setContentsMargins(0, 0, 0, 0)

        # Buttons
        self.editButton = QToolButton()
        self.reRecogButton = QToolButton()
        self.reRecogButton.setIcon(newIcon('reRec', 30))
        self.reRecogButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)

        self.tableRecButton = QToolButton()
        self.tableRecButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)

        self.newButton = QToolButton()
        self.newButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)
        self.createpolyButton = QToolButton()
        self.createpolyButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)

        self.SaveButton = QToolButton()
        self.SaveButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)
        self.DelButton = QToolButton()
        self.DelButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)

        leftTopToolBox = QGridLayout()
        leftTopToolBox.addWidget(self.newButton, 0, 0, 1, 1)
        leftTopToolBox.addWidget(self.createpolyButton, 0, 1, 1, 1)
        leftTopToolBox.addWidget(self.reRecogButton, 1, 0, 1, 1)
        leftTopToolBox.addWidget(self.tableRecButton, 1, 1, 1, 1)

        leftTopToolBoxContainer = QWidget()
        leftTopToolBoxContainer.setLayout(leftTopToolBox)
        listLayout.addWidget(leftTopToolBoxContainer)

        #  ================== Label List  ==================
        labelIndexListlBox = QHBoxLayout()

        # Create and add a widget for showing current label item index
        self.indexList = QListWidget()
        self.indexList.setMaximumSize(30, 16777215) # limit max width
        self.indexList.setEditTriggers(QAbstractItemView.NoEditTriggers) # no editable
        self.indexList.itemSelectionChanged.connect(self.indexSelectionChanged)
        self.indexList.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff) # no scroll Bar
        self.indexListDock = QDockWidget('No.', self)
        self.indexListDock.setWidget(self.indexList)
        self.indexListDock.setFeatures(QDockWidget.NoDockWidgetFeatures)
        labelIndexListlBox.addWidget(self.indexListDock, 1)
        # no margin between two boxes
        labelIndexListlBox.setSpacing(0)
        
        # Create and add a widget for showing current label items
        self.labelList = EditInList()
        labelListContainer = QWidget()
        labelListContainer.setLayout(listLayout)
        self.labelList.itemSelectionChanged.connect(self.labelSelectionChanged)
        self.labelList.clicked.connect(self.labelList.item_clicked)

        # Connect to itemChanged to detect checkbox changes.
        self.labelList.itemChanged.connect(self.labelItemChanged)
        self.labelListDockName = getStr('recognitionResult')
        self.labelListDock = QDockWidget(self.labelListDockName, self)
        self.labelListDock.setWidget(self.labelList)
        self.labelListDock.setFeatures(QDockWidget.NoDockWidgetFeatures)
        labelIndexListlBox.addWidget(self.labelListDock, 10) # label list is wider than index list
        
        # enable labelList drag_drop to adjust bbox order
        # 设置选择模式为单选  
        self.labelList.setSelectionMode(QAbstractItemView.SingleSelection)
        # 启用拖拽
        self.labelList.setDragEnabled(True)
        # 设置接受拖放
        self.labelList.viewport().setAcceptDrops(True)
        # 设置显示将要被放置的位置
        self.labelList.setDropIndicatorShown(True)
        # 设置拖放模式为移动项目，如果不设置，默认为复制项目
        self.labelList.setDragDropMode(QAbstractItemView.InternalMove) 
        # 触发放置
        self.labelList.model().rowsMoved.connect(self.drag_drop_happened)

        labelIndexListContainer = QWidget()
        labelIndexListContainer.setLayout(labelIndexListlBox)
        listLayout.addWidget(labelIndexListContainer)

        # labelList indexList同步滚动
        self.labelListBar = self.labelList.verticalScrollBar()
        self.indexListBar = self.indexList.verticalScrollBar()

        self.labelListBar.valueChanged.connect(self.move_scrollbar)
        self.indexListBar.valueChanged.connect(self.move_scrollbar)

        #  ================== Detection Box  ==================
        self.BoxList = QListWidget()

        # self.BoxList.itemActivated.connect(self.boxSelectionChanged)
        self.BoxList.itemSelectionChanged.connect(self.boxSelectionChanged)
        self.BoxList.itemDoubleClicked.connect(self.editBox)
        # Connect to itemChanged to detect checkbox changes.
        self.BoxList.itemChanged.connect(self.boxItemChanged)
        self.BoxListDockName = getStr('detectionBoxposition')
        self.BoxListDock = QDockWidget(self.BoxListDockName, self)
        self.BoxListDock.setWidget(self.BoxList)
        self.BoxListDock.setFeatures(QDockWidget.NoDockWidgetFeatures)
        listLayout.addWidget(self.BoxListDock)

        #  ================== Lower Right Area  ==================
        leftbtmtoolbox = QHBoxLayout()
        leftbtmtoolbox.addWidget(self.SaveButton)
        leftbtmtoolbox.addWidget(self.DelButton)
        leftbtmtoolboxcontainer = QWidget()
        leftbtmtoolboxcontainer.setLayout(leftbtmtoolbox)
        listLayout.addWidget(leftbtmtoolboxcontainer)

        self.dock = QDockWidget(getStr('boxLabelText'), self)
        self.dock.setObjectName(getStr('labels'))
        self.dock.setWidget(labelListContainer)

        #  ================== Zoom Bar  ==================
        self.imageSlider = QSlider(Qt.Horizontal)
        self.imageSlider.valueChanged.connect(self.CanvasSizeChange)
        self.imageSlider.setMinimum(-9)
        self.imageSlider.setMaximum(510)
        self.imageSlider.setSingleStep(1)
        self.imageSlider.setTickPosition(QSlider.TicksBelow)
        self.imageSlider.setTickInterval(1)

        op = QGraphicsOpacityEffect()
        op.setOpacity(0.2)
        self.imageSlider.setGraphicsEffect(op)

        self.imageSlider.setStyleSheet("background-color:transparent")
        self.imageSliderDock = QDockWidget(getStr('ImageResize'), self)
        self.imageSliderDock.setObjectName(getStr('IR'))
        self.imageSliderDock.setWidget(self.imageSlider)
        self.imageSliderDock.setFeatures(QDockWidget.DockWidgetFloatable)
        self.imageSliderDock.setAttribute(Qt.WA_TranslucentBackground)
        self.addDockWidget(Qt.RightDockWidgetArea, self.imageSliderDock)

        self.zoomWidget = ZoomWidget()
        self.colorDialog = ColorDialog(parent=self)
        self.zoomWidgetValue = self.zoomWidget.value()

        self.msgBox = QMessageBox()

        #  ================== Thumbnail ==================
        hlayout = QHBoxLayout()
        m = (0, 0, 0, 0)
        hlayout.setSpacing(0)
        hlayout.setContentsMargins(*m)
        self.preButton = QToolButton()
        self.preButton.setIcon(newIcon("prev", 40))
        self.preButton.setIconSize(QSize(40, 100))
        self.preButton.clicked.connect(self.openPrevImg)
        self.preButton.setStyleSheet('border: none;')
        self.preButton.setShortcut('a')
        self.iconlist = QListWidget()
        self.iconlist.setViewMode(QListView.IconMode)
        self.iconlist.setFlow(QListView.TopToBottom)
        self.iconlist.setSpacing(10)
        self.iconlist.setIconSize(QSize(50, 50))
        self.iconlist.setMovement(QListView.Static)
        self.iconlist.setResizeMode(QListView.Adjust)
        self.iconlist.itemClicked.connect(self.iconitemDoubleClicked)
        self.iconlist.setStyleSheet("QListWidget{ background-color:transparent; border: none;}")
        self.iconlist.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        self.nextButton = QToolButton()
        self.nextButton.setIcon(newIcon("next", 40))
        self.nextButton.setIconSize(QSize(40, 100))
        self.nextButton.setStyleSheet('border: none;')
        self.nextButton.clicked.connect(self.openNextImg)
        self.nextButton.setShortcut('d')

        hlayout.addWidget(self.preButton)
        hlayout.addWidget(self.iconlist)
        hlayout.addWidget(self.nextButton)

        iconListContainer = QWidget()
        iconListContainer.setLayout(hlayout)
        iconListContainer.setFixedHeight(100)

        #  ================== Canvas ==================
        self.canvas = Canvas(parent=self)
        self.canvas.zoomRequest.connect(self.zoomRequest)
        self.canvas.setDrawingShapeToSquare(settings.get(SETTING_DRAW_SQUARE, False))

        scroll = QScrollArea()
        scroll.setWidget(self.canvas)
        scroll.setWidgetResizable(True)
        self.scrollBars = {
            Qt.Vertical: scroll.verticalScrollBar(),
            Qt.Horizontal: scroll.horizontalScrollBar()
        }
        self.scrollArea = scroll
        self.canvas.scrollRequest.connect(self.scrollRequest)

        self.canvas.newShape.connect(partial(self.newShape, False))
        self.canvas.shapeMoved.connect(self.updateBoxlist)  # self.setDirty
        self.canvas.selectionChanged.connect(self.shapeSelectionChanged)
        self.canvas.drawingPolygon.connect(self.toggleDrawingSensitive)

        centerLayout = QVBoxLayout()
        centerLayout.setContentsMargins(0, 0, 0, 0)
        centerLayout.addWidget(scroll)
        centerLayout.addWidget(iconListContainer, 0, Qt.AlignCenter)
        centerContainer = QWidget()
        centerContainer.setLayout(centerLayout)

        self.setCentralWidget(centerContainer)
        self.addDockWidget(Qt.RightDockWidgetArea, self.dock)

        self.dock.setFeatures(QDockWidget.DockWidgetClosable | QDockWidget.DockWidgetFloatable)
        self.fileDock.setFeatures(QDockWidget.NoDockWidgetFeatures)

        #  ================== Actions ==================
        action = partial(newAction, self)
        quit = action(getStr('quit'), self.close,
                      'Ctrl+Q', 'quit', getStr('quitApp'))

        opendir = action(getStr('openDir'), self.openDirDialog,
                         'Ctrl+u', 'open', getStr('openDir'))

        open_dataset_dir = action(getStr('openDatasetDir'), self.openDatasetDirDialog,
                                  'Ctrl+p', 'open', getStr('openDatasetDir'), enabled=False)

        save = action(getStr('save'), self.saveFile,
                      ['Ctrl+V','د'], 'verify', getStr('saveDetail'), enabled=True)

        alcm = action(getStr('choosemodel'), self.autolcm,
                      'Ctrl+M', 'next', getStr('tipchoosemodel'))

        deleteImg = action(getStr('deleteImg'), self.deleteImg, 'Ctrl+Shift+D', 'close', getStr('deleteImgDetail'),
                           enabled=True)

        resetAll = action(getStr('resetAll'), self.resetAll, None, 'resetall', getStr('resetAllDetail'))

        color1 = action(getStr('boxLineColor'), self.chooseColor,
                        'Ctrl+L', 'color_line', getStr('boxLineColorDetail'))

        createMode = action(getStr('crtBox'), self.setCreateMode,
                            ['w', 'ص'], 'new', getStr('crtBoxDetail'), enabled=True)
        editMode = action('&Edit\nRectBox', self.setEditMode,
                          'Ctrl+J', 'edit', u'Move and edit Boxs', enabled=False)

        create = action(getStr('crtBox'), self.createShape,
                        ['w', 'ص'], 'objects', getStr('crtBoxDetail'), enabled=True)

        delete = action(getStr('delBox'), self.deleteSelectedShape,
                        'backspace', 'delete', getStr('delBoxDetail'), enabled=False)

        copy = action(getStr('dupBox'), self.copySelectedShape,
                      'Ctrl+C', 'copy', getStr('dupBoxDetail'),
                      enabled=False)

        hideAll = action(getStr('hideBox'), partial(self.togglePolygons, False),
                         'Ctrl+H', 'hide', getStr('hideAllBoxDetail'),
                         enabled=False)
        showAll = action(getStr('showBox'), partial(self.togglePolygons, True),
                         'Ctrl+A', 'hide', getStr('showAllBoxDetail'),
                         enabled=False)

        help = action(getStr('tutorial'), self.showTutorialDialog, None, 'help', getStr('tutorialDetail'))
        showInfo = action(getStr('info'), self.showInfoDialog, None, 'help', getStr('info'))
        showSteps = action(getStr('steps'), self.showStepsDialog, None, 'help', getStr('steps'))
        showKeys = action(getStr('keys'), self.showKeysDialog, None, 'help', getStr('keys'))

        zoom = QWidgetAction(self)
        zoom.setDefaultWidget(self.zoomWidget)
        self.zoomWidget.setWhatsThis(
            u"Zoom in or out of the image. Also accessible with"
            " %s and %s from the canvas." % (fmtShortcut("Ctrl+[-+]"),
                                             fmtShortcut("Ctrl+Wheel")))
        self.zoomWidget.setEnabled(False)

        zoomIn = action(getStr('zoomin'), partial(self.addZoom, 10),
                        'Ctrl++', 'zoom-in', getStr('zoominDetail'), enabled=False)
        zoomOut = action(getStr('zoomout'), partial(self.addZoom, -10),
                         'Ctrl+-', 'zoom-out', getStr('zoomoutDetail'), enabled=False)
        zoomOrg = action(getStr('originalsize'), partial(self.setZoom, 100),
                         'Ctrl+=', 'zoom', getStr('originalsizeDetail'), enabled=False)
        fitWindow = action(getStr('fitWin'), self.setFitWindow,
                           'Ctrl+F', 'fit-window', getStr('fitWinDetail'),
                           checkable=True, enabled=False)
        fitWidth = action(getStr('fitWidth'), self.setFitWidth,
                          'Ctrl+Shift+F', 'fit-width', getStr('fitWidthDetail'),
                          checkable=True, enabled=False)
        # Group zoom controls into a list for easier toggling.
        zoomActions = (self.zoomWidget, zoomIn, zoomOut,
                       zoomOrg, fitWindow, fitWidth)
        self.zoomMode = self.MANUAL_ZOOM
        self.scalers = {
            self.FIT_WINDOW: self.scaleFitWindow,
            self.FIT_WIDTH: self.scaleFitWidth,
            # Set to one to scale to 100% when loading files.
            self.MANUAL_ZOOM: lambda: 1,
        }

        #  ================== New Actions ==================

        edit = action(getStr('editLabel'), self.editLabel,
                      'Ctrl+E', 'edit', getStr('editLabelDetail'), enabled=False)

        AutoRec = action(getStr('autoRecognition'), self.autoRecognition,
                         '', 'Auto', getStr('autoRecognition'), enabled=False)

        reRec = action(getStr('reRecognition'), self.reRecognition,
                       'Ctrl+Shift+R', 'reRec', getStr('reRecognition'), enabled=False)

        singleRere = action(getStr('singleRe'), self.singleRerecognition,
                            'Ctrl+R', 'reRec', getStr('singleRe'), enabled=False)

        createpoly = action(getStr('creatPolygon'), self.createPolygon,
                            'q', 'new', getStr('creatPolygon'), enabled=False)
        
        tableRec = action(getStr('TableRecognition'), self.TableRecognition,
                        '', 'Auto', getStr('TableRecognition'), enabled=False)

        cellreRec = action(getStr('cellreRecognition'), self.cellreRecognition,
                        '', 'reRec', getStr('cellreRecognition'), enabled=False)

        saveRec = action(getStr('saveRec'), self.saveRecResult,
                         '', 'save', getStr('saveRec'), enabled=False)

        saveLabel = action(getStr('saveLabel'), self.saveLabelFile,  #
                           'Ctrl+S', 'save', getStr('saveLabel'), enabled=False)
        
        exportJSON = action(getStr('exportJSON'), self.exportJSON,
                            '', 'save', getStr('exportJSON'), enabled=False)

        undoLastPoint = action(getStr("undoLastPoint"), self.canvas.undoLastPoint,
                               'Ctrl+Z', "undo", getStr("undoLastPoint"), enabled=False)

        rotateLeft = action(getStr("rotateLeft"), partial(self.rotateImgAction, 1),
                            'Ctrl+Alt+L', "rotateLeft", getStr("rotateLeft"), enabled=False)

        rotateRight = action(getStr("rotateRight"), partial(self.rotateImgAction, -1),
                             'Ctrl+Alt+R', "rotateRight", getStr("rotateRight"), enabled=False)

        undo = action(getStr("undo"), self.undoShapeEdit,
                      'Ctrl+Z', "undo", getStr("undo"), enabled=False)

        change_cls = action(getStr("keyChange"), self.change_box_key,
                            'Ctrl+X', "edit", getStr("keyChange"), enabled=False)

        lock = action(getStr("lockBox"), self.lockSelectedShape,
                      None, "lock", getStr("lockBoxDetail"), enabled=False)

        self.editButton.setDefaultAction(edit)
        self.newButton.setDefaultAction(create)
        self.createpolyButton.setDefaultAction(createpoly)
        self.DelButton.setDefaultAction(deleteImg)
        self.SaveButton.setDefaultAction(save)
        self.AutoRecognition.setDefaultAction(AutoRec)
        self.reRecogButton.setDefaultAction(reRec)
        self.tableRecButton.setDefaultAction(tableRec)
        # self.preButton.setDefaultAction(openPrevImg)
        # self.nextButton.setDefaultAction(openNextImg)

        #  ================== Zoom layout ==================
        zoomLayout = QHBoxLayout()
        zoomLayout.addStretch()
        self.zoominButton = QToolButton()
        self.zoominButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)
        self.zoominButton.setDefaultAction(zoomIn)
        self.zoomoutButton = QToolButton()
        self.zoomoutButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)
        self.zoomoutButton.setDefaultAction(zoomOut)
        self.zoomorgButton = QToolButton()
        self.zoomorgButton.setToolButtonStyle(Qt.ToolButtonTextBesideIcon)
        self.zoomorgButton.setDefaultAction(zoomOrg)
        zoomLayout.addWidget(self.zoominButton)
        zoomLayout.addWidget(self.zoomorgButton)
        zoomLayout.addWidget(self.zoomoutButton)

        zoomContainer = QWidget()
        zoomContainer.setLayout(zoomLayout)
        zoomContainer.setGeometry(0, 0, 30, 150)

        shapeLineColor = action(getStr('shapeLineColor'), self.chshapeLineColor,
                                icon='color_line', tip=getStr('shapeLineColorDetail'),
                                enabled=False)
        shapeFillColor = action(getStr('shapeFillColor'), self.chshapeFillColor,
                                icon='color', tip=getStr('shapeFillColorDetail'),
                                enabled=False)

        # Label list context menu.
        labelMenu = QMenu()
        addActions(labelMenu, (edit, delete))

        self.labelList.setContextMenuPolicy(Qt.CustomContextMenu)
        self.labelList.customContextMenuRequested.connect(self.popLabelListMenu)

        # Draw squares/rectangles
        self.drawSquaresOption = QAction(getStr('drawSquares'), self)
        self.drawSquaresOption.setCheckable(True)
        self.drawSquaresOption.setChecked(settings.get(SETTING_DRAW_SQUARE, False))
        self.drawSquaresOption.triggered.connect(self.toogleDrawSquare)

        # Store actions for further handling.
        self.actions = struct(save=save, resetAll=resetAll, deleteImg=deleteImg,
                              lineColor=color1, create=create, createpoly=createpoly, tableRec=tableRec, delete=delete, edit=edit, copy=copy,
                              saveRec=saveRec, singleRere=singleRere, AutoRec=AutoRec, reRec=reRec, cellreRec=cellreRec,
                              createMode=createMode, editMode=editMode,
                              shapeLineColor=shapeLineColor, shapeFillColor=shapeFillColor,
                              zoom=zoom, zoomIn=zoomIn, zoomOut=zoomOut, zoomOrg=zoomOrg,
                              fitWindow=fitWindow, fitWidth=fitWidth,
                              zoomActions=zoomActions, saveLabel=saveLabel, change_cls=change_cls,
                              undo=undo, undoLastPoint=undoLastPoint, open_dataset_dir=open_dataset_dir,
                              rotateLeft=rotateLeft, rotateRight=rotateRight, lock=lock, exportJSON=exportJSON,
                              fileMenuActions=(opendir, open_dataset_dir, saveLabel, exportJSON, resetAll, quit),
                              beginner=(), advanced=(),
                              editMenu=(createpoly, edit, copy, delete, singleRere, cellreRec, None, undo, undoLastPoint,
                                        None, rotateLeft, rotateRight, None, color1, self.drawSquaresOption, lock,
                                        None, change_cls),
                              beginnerContext=(
                                  create, createpoly, edit, copy, delete, singleRere, cellreRec, rotateLeft, rotateRight, lock, change_cls),
                              advancedContext=(createMode, editMode, edit, copy,
                                               delete, shapeLineColor, shapeFillColor),
                              onLoadActive=(create, createpoly, createMode, editMode),
                              onShapesPresent=(hideAll, showAll))

        # menus
        self.menus = struct(
            file=self.menu('&' + getStr('mfile')),
            edit=self.menu('&' + getStr('medit')),
            view=self.menu('&' + getStr('mview')),
            autolabel=self.menu('&PaddleOCR'),
            help=self.menu('&' + getStr('mhelp')),
            recentFiles=QMenu('Open &Recent'),
            labelList=labelMenu)

        self.lastLabel = None
        # Add option to enable/disable labels being displayed at the top of bounding boxes
        self.displayLabelOption = QAction(getStr('displayLabel'), self)
        self.displayLabelOption.setShortcut("Ctrl+Shift+P")
        self.displayLabelOption.setCheckable(True)
        self.displayLabelOption.setChecked(settings.get(SETTING_PAINT_LABEL, False))
        self.displayLabelOption.triggered.connect(self.togglePaintLabelsOption)

        # Add option to enable/disable box index being displayed at the top of bounding boxes
        self.displayIndexOption = QAction(getStr('displayIndex'), self)
        self.displayIndexOption.setCheckable(True)
        self.displayIndexOption.setChecked(settings.get(SETTING_PAINT_INDEX, False))
        self.displayIndexOption.triggered.connect(self.togglePaintIndexOption)

        self.labelDialogOption = QAction(getStr('labelDialogOption'), self)
        self.labelDialogOption.setShortcut("Ctrl+Shift+L")
        self.labelDialogOption.setCheckable(True)
        self.labelDialogOption.setChecked(settings.get(SETTING_PAINT_LABEL, False))
        self.displayIndexOption.setChecked(settings.get(SETTING_PAINT_INDEX, False))
        self.labelDialogOption.triggered.connect(self.speedChoose)

        self.autoSaveOption = QAction(getStr('autoSaveMode'), self)
        self.autoSaveOption.setCheckable(True)
        self.autoSaveOption.setChecked(settings.get(SETTING_PAINT_LABEL, False))
        self.displayIndexOption.setChecked(settings.get(SETTING_PAINT_INDEX, False))
        self.autoSaveOption.triggered.connect(self.autoSaveFunc)

        addActions(self.menus.file,
                   (opendir, open_dataset_dir, None, saveLabel, saveRec, exportJSON, self.autoSaveOption, None, resetAll, deleteImg,
                    quit))

        addActions(self.menus.help, (showKeys, showSteps, showInfo))
        addActions(self.menus.view, (
            self.displayLabelOption, self.displayIndexOption, self.labelDialogOption,
            None,
            hideAll, showAll, None,
            zoomIn, zoomOut, zoomOrg, None,
            fitWindow, fitWidth))

        addActions(self.menus.autolabel, (AutoRec, reRec, cellreRec, alcm, None, help))

        self.menus.file.aboutToShow.connect(self.updateFileMenu)

        # Custom context menu for the canvas widget:
        addActions(self.canvas.menus[0], self.actions.beginnerContext)

        self.statusBar().showMessage('%s started.' % __appname__)
        self.statusBar().show()

        # Application state.
        self.image = QImage()
        self.filePath = ustr(default_filename)
        self.lastOpenDir = None
        self.recentFiles = []
        self.maxRecent = 7
        self.lineColor = None
        self.fillColor = None
        self.zoom_level = 100
        self.fit_window = False
        # Add Chris
        self.difficult = False

        # Fix the compatible issue for qt4 and qt5. Convert the QStringList to python list
        if settings.get(SETTING_RECENT_FILES):
            if have_qstring():
                recentFileQStringList = settings.get(SETTING_RECENT_FILES)
                self.recentFiles = [ustr(i) for i in recentFileQStringList]
            else:
                self.recentFiles = recentFileQStringList = settings.get(SETTING_RECENT_FILES)

        size = settings.get(SETTING_WIN_SIZE, QSize(1200, 800))

        position = QPoint(0, 0)
        saved_position = settings.get(SETTING_WIN_POSE, position)
        # Fix the multiple monitors issue
        for i in range(QApplication.desktop().screenCount()):
            if QApplication.desktop().availableGeometry(i).contains(saved_position):
                position = saved_position
                break
        self.resize(size)
        self.move(position)
        saveDir = ustr(settings.get(SETTING_SAVE_DIR, None))
        self.lastOpenDir = ustr(settings.get(SETTING_LAST_OPEN_DIR, None))

        self.restoreState(settings.get(SETTING_WIN_STATE, QByteArray()))
        Shape.line_color = self.lineColor = QColor(settings.get(SETTING_LINE_COLOR, DEFAULT_LINE_COLOR))
        Shape.fill_color = self.fillColor = QColor(settings.get(SETTING_FILL_COLOR, DEFAULT_FILL_COLOR))
        self.canvas.setDrawingColor(self.lineColor)
        # Add chris
        Shape.difficult = self.difficult

        # ADD:
        # Populate the File menu dynamically.
        self.updateFileMenu()

        # Since loading the file may take some time, make sure it runs in the background.
        if self.filePath and os.path.isdir(self.filePath):
            self.queueEvent(partial(self.importDirImages, self.filePath or ""))
        elif self.filePath:
            self.queueEvent(partial(self.loadFile, self.filePath or ""))

        self.keyDialog = None

        # Callbacks:
        self.zoomWidget.valueChanged.connect(self.paintCanvas)

        self.populateModeActions()

        # Display cursor coordinates at the right of status bar
        self.labelCoordinates = QLabel('')
        self.statusBar().addPermanentWidget(self.labelCoordinates)

        # Open Dir if deafult file
        if self.filePath and os.path.isdir(self.filePath):
            self.openDirDialog(dirpath=self.filePath, silent=True)

    def menu(self, title, actions=None):
        menu = self.menuBar().addMenu(title)
        if actions:
            addActions(menu, actions)
        return menu

    def keyReleaseEvent(self, event):
        if event.key() == Qt.Key_Control:
            self.canvas.setDrawingShapeToSquare(False)

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Control:
            # Draw rectangle if Ctrl is pressed
            self.canvas.setDrawingShapeToSquare(True)

    def noShapes(self):
        return not self.itemsToShapes

    def populateModeActions(self):
        self.canvas.menus[0].clear()
        addActions(self.canvas.menus[0], self.actions.beginnerContext)
        self.menus.edit.clear()
        actions = (self.actions.create,)  # if self.beginner() else (self.actions.createMode, self.actions.editMode)
        addActions(self.menus.edit, actions + self.actions.editMenu)

    def setDirty(self):
        self.dirty = True
        self.actions.save.setEnabled(True)

    def setClean(self):
        self.dirty = False
        self.actions.save.setEnabled(False)
        self.actions.create.setEnabled(True)
        self.actions.createpoly.setEnabled(True)

    def toggleActions(self, value=True):
        """Enable/Disable widgets which depend on an opened image."""
        for z in self.actions.zoomActions:
            z.setEnabled(value)
        for action in self.actions.onLoadActive:
            action.setEnabled(value)

    def queueEvent(self, function):
        QTimer.singleShot(0, function)

    def status(self, message, delay=5000):
        self.statusBar().showMessage(message, delay)

    def resetState(self):
        self.itemsToShapes.clear()
        self.shapesToItems.clear()
        self.itemsToShapesbox.clear()  # ADD
        self.shapesToItemsbox.clear()
        self.labelList.clear()
        self.BoxList.clear()
        self.indexList.clear()
        self.filePath = None
        self.imageData = None
        self.labelFile = None
        self.canvas.resetState()
        self.labelCoordinates.clear()
        # self.comboBox.cb.clear()
        self.result_dic = []

    def currentItem(self):
        items = self.labelList.selectedItems()
        if items:
            return items[0]
        return None

    def currentBox(self):
        items = self.BoxList.selectedItems()
        if items:
            return items[0]
        return None

    def addRecentFile(self, filePath):
        if filePath in self.recentFiles:
            self.recentFiles.remove(filePath)
        elif len(self.recentFiles) >= self.maxRecent:
            self.recentFiles.pop()
        self.recentFiles.insert(0, filePath)

    def beginner(self):
        return self._beginner

    def advanced(self):
        return not self.beginner()

    def getAvailableScreencastViewer(self):
        osName = platform.system()

        if osName == 'Windows':
            return ['C:\\Program Files\\Internet Explorer\\iexplore.exe']
        elif osName == 'Linux':
            return ['xdg-open']
        elif osName == 'Darwin':
            return ['open']

    ## Callbacks ##
    def showTutorialDialog(self):
        subprocess.Popen(self.screencastViewer + [self.screencast])

    def showInfoDialog(self):
        from libs.__init__ import __version__
        msg = u'Name:{0} \nApp Version:{1} \n{2} '.format(__appname__, __version__, sys.version_info)
        QMessageBox.information(self, u'Information', msg)

    def showStepsDialog(self):
        msg = stepsInfo(self.lang)
        QMessageBox.information(self, u'Information', msg)

    def showKeysDialog(self):
        msg = keysInfo(self.lang)
        QMessageBox.information(self, u'Information', msg)

    def createShape(self):
        assert self.beginner()
        self.canvas.setEditing(False)
        self.actions.create.setEnabled(False)
        self.actions.createpoly.setEnabled(False)
        self.canvas.fourpoint = False

    def createPolygon(self):
        assert self.beginner()
        self.canvas.setEditing(False)
        self.canvas.fourpoint = True
        self.actions.create.setEnabled(False)
        self.actions.createpoly.setEnabled(False)
        self.actions.undoLastPoint.setEnabled(True)

    def rotateImg(self, filename, k, _value):
        self.actions.rotateRight.setEnabled(_value)
        pix = cv2.imread(filename)
        pix = np.rot90(pix, k)
        cv2.imwrite(filename, pix)
        self.canvas.update()
        self.loadFile(filename)

    def rotateImgWarn(self):
        if self.lang == 'ch':
            self.msgBox.warning(self, "提示", "\n 该图片已经有标注框,旋转操作会打乱标注,建议清除标注框后旋转。")
        else:
            self.msgBox.warning(self, "Warn", "\n The picture already has a label box, "
                                              "and rotation will disrupt the label. "
                                              "It is recommended to clear the label box and rotate it.")

    def rotateImgAction(self, k=1, _value=False):

        filename = self.mImgList[self.currIndex]

        if os.path.exists(filename):
            if self.itemsToShapesbox:
                self.rotateImgWarn()
            else:
                self.saveFile()
                self.dirty = False
                self.rotateImg(filename=filename, k=k, _value=True)
        else:
            self.rotateImgWarn()
            self.actions.rotateRight.setEnabled(False)
            self.actions.rotateLeft.setEnabled(False)

    def toggleDrawingSensitive(self, drawing=True):
        """In the middle of drawing, toggling between modes should be disabled."""
        self.actions.editMode.setEnabled(not drawing)
        if not drawing and self.beginner():
            # Cancel creation.
            print('Cancel creation.')
            self.canvas.setEditing(True)
            self.canvas.restoreCursor()
            self.actions.create.setEnabled(True)
            self.actions.createpoly.setEnabled(True)

    def toggleDrawMode(self, edit=True):
        self.canvas.setEditing(edit)
        self.actions.createMode.setEnabled(edit)
        self.actions.editMode.setEnabled(not edit)

    def setCreateMode(self):
        assert self.advanced()
        self.toggleDrawMode(False)

    def setEditMode(self):
        assert self.advanced()
        self.toggleDrawMode(True)
        self.labelSelectionChanged()

    def updateFileMenu(self):
        currFilePath = self.filePath

        def exists(filename):
            return os.path.exists(filename)

        menu = self.menus.recentFiles
        menu.clear()
        files = [f for f in self.recentFiles if f !=
                 currFilePath and exists(f)]
        for i, f in enumerate(files):
            icon = newIcon('labels')
            action = QAction(
                icon, '&%d %s' % (i + 1, QFileInfo(f).fileName()), self)
            action.triggered.connect(partial(self.loadRecent, f))
            menu.addAction(action)

    def popLabelListMenu(self, point):
        self.menus.labelList.exec_(self.labelList.mapToGlobal(point))

    def editLabel(self):
        if not self.canvas.editing():
            return
        item = self.currentItem()
        if not item:
            return
        text = self.labelDialog.popUp(item.text())
        if text is not None:
            item.setText(text)
            # item.setBackground(generateColorByText(text))
            self.setDirty()
            self.updateComboBox()

    # =================== detection box related functions ===================
    def boxItemChanged(self, item):
        shape = self.itemsToShapesbox[item]

        box = ast.literal_eval(item.text())
        # print('shape in labelItemChanged is',shape.points)
        if box != [(int(p.x()), int(p.y())) for p in shape.points]:
            # shape.points = box
            shape.points = [QPointF(p[0], p[1]) for p in box]

            # QPointF(x,y)
            # shape.line_color = generateColorByText(shape.label)
            self.setDirty()
        else:  # User probably changed item visibility
            self.canvas.setShapeVisible(shape, True)  # item.checkState() == Qt.Checked

    def editBox(self):  # ADD
        if not self.canvas.editing():
            return
        item = self.currentBox()
        if not item:
            return
        text = self.labelDialog.popUp(item.text())

        imageSize = str(self.image.size())
        width, height = self.image.width(), self.image.height()
        if text:
            try:
                text_list = eval(text)
            except:
                msg_box = QMessageBox(QMessageBox.Warning, 'Warning', 'Please enter the correct format')
                msg_box.exec_()
                return
            if len(text_list) < 4:
                msg_box = QMessageBox(QMessageBox.Warning, 'Warning', 'Please enter the coordinates of 4 points')
                msg_box.exec_()
                return
            for box in text_list:
                if box[0] > width or box[0] < 0 or box[1] > height or box[1] < 0:
                    msg_box = QMessageBox(QMessageBox.Warning, 'Warning', 'Out of picture size')
                    msg_box.exec_()
                    return

            item.setText(text)
            # item.setBackground(generateColorByText(text))
            self.setDirty()
            self.updateComboBox()

    def updateBoxlist(self):
        self.canvas.selectedShapes_hShape = []
        if self.canvas.hShape != None:
            self.canvas.selectedShapes_hShape = self.canvas.selectedShapes + [self.canvas.hShape]
        else:
            self.canvas.selectedShapes_hShape = self.canvas.selectedShapes
        for shape in self.canvas.selectedShapes_hShape:
            if shape in self.shapesToItemsbox.keys():
                item = self.shapesToItemsbox[shape]  # listitem
                text = [(int(p.x()), int(p.y())) for p in shape.points]
                item.setText(str(text))
        self.actions.undo.setEnabled(True)
        self.setDirty()

    def indexTo5Files(self, currIndex):
        if currIndex < 2:
            return self.mImgList[:5]
        elif currIndex > len(self.mImgList) - 3:
            return self.mImgList[-5:]
        else:
            return self.mImgList[currIndex - 2: currIndex + 3]

    # Tzutalin 20160906 : Add file list and dock to move faster
    def fileitemDoubleClicked(self, item=None):
        self.currIndex = self.mImgList.index(ustr(os.path.join(os.path.abspath(self.dirname), item.text())))
        filename = self.mImgList[self.currIndex]
        if filename:
            self.mImgList5 = self.indexTo5Files(self.currIndex)
            # self.additems5(None)
            self.loadFile(filename)

    def iconitemDoubleClicked(self, item=None):
        self.currIndex = self.mImgList.index(ustr(os.path.join(item.toolTip())))
        filename = self.mImgList[self.currIndex]
        if filename:
            self.mImgList5 = self.indexTo5Files(self.currIndex)
            # self.additems5(None)
            self.loadFile(filename)

    def CanvasSizeChange(self):
        if len(self.mImgList) > 0 and self.imageSlider.hasFocus():
            self.zoomWidget.setValue(self.imageSlider.value())

    def shapeSelectionChanged(self, selected_shapes):
        self._noSelectionSlot = True
        for shape in self.canvas.selectedShapes:
            shape.selected = False
        self.labelList.clearSelection()
        self.indexList.clearSelection()
        self.canvas.selectedShapes = selected_shapes
        for shape in self.canvas.selectedShapes:
            shape.selected = True
            self.shapesToItems[shape].setSelected(True)
            self.shapesToItemsbox[shape].setSelected(True)
            index = self.labelList.indexFromItem(self.shapesToItems[shape]).row()
            self.indexList.item(index).setSelected(True)

        self.labelList.scrollToItem(self.currentItem())  # QAbstractItemView.EnsureVisible
        # map current label item to index item and select it
        index = self.labelList.indexFromItem(self.currentItem()).row()
        self.indexList.scrollToItem(self.indexList.item(index)) 
        self.BoxList.scrollToItem(self.currentBox())

        if self.kie_mode:
            if len(self.canvas.selectedShapes) == 1 and self.keyList.count() > 0:
                selected_key_item_row = self.keyList.findItemsByLabel(self.canvas.selectedShapes[0].key_cls,
                                                                      get_row=True)
                if isinstance(selected_key_item_row, list) and len(selected_key_item_row) == 0:
                    key_text = self.canvas.selectedShapes[0].key_cls
                    item = self.keyList.createItemFromLabel(key_text)
                    self.keyList.addItem(item)
                    rgb = self._get_rgb_by_label(key_text, self.kie_mode)
                    self.keyList.setItemLabel(item, key_text, rgb)
                    selected_key_item_row = self.keyList.findItemsByLabel(self.canvas.selectedShapes[0].key_cls,
                                                                          get_row=True)

                self.keyList.setCurrentRow(selected_key_item_row)

        self._noSelectionSlot = False
        n_selected = len(selected_shapes)
        self.actions.singleRere.setEnabled(n_selected)
        self.actions.cellreRec.setEnabled(n_selected)
        self.actions.delete.setEnabled(n_selected)
        self.actions.copy.setEnabled(n_selected)
        self.actions.edit.setEnabled(n_selected == 1)
        self.actions.lock.setEnabled(n_selected)
        self.actions.change_cls.setEnabled(n_selected)

    def addLabel(self, shape):
        shape.paintLabel = self.displayLabelOption.isChecked()
        shape.paintIdx = self.displayIndexOption.isChecked()

        item = HashableQListWidgetItem(shape.label)
        # current difficult checkbox is disenble
        # item.setFlags(item.flags() | Qt.ItemIsUserCheckable)
        # item.setCheckState(Qt.Unchecked) if shape.difficult else item.setCheckState(Qt.Checked)

        # Checked means difficult is False
        # item.setBackground(generateColorByText(shape.label))
        self.itemsToShapes[item] = shape
        self.shapesToItems[shape] = item
        # add current label item index before label string
        current_index = QListWidgetItem(str(self.labelList.count()))
        current_index.setTextAlignment(Qt.AlignHCenter)
        self.indexList.addItem(current_index)
        self.labelList.addItem(item)
        # print('item in add label is ',[(p.x(), p.y()) for p in shape.points], shape.label)

        # ADD for box
        item = HashableQListWidgetItem(str([(int(p.x()), int(p.y())) for p in shape.points]))
        self.itemsToShapesbox[item] = shape
        self.shapesToItemsbox[shape] = item
        self.BoxList.addItem(item)
        for action in self.actions.onShapesPresent:
            action.setEnabled(True)
        self.updateComboBox()

        # update show counting
        self.BoxListDock.setWindowTitle(self.BoxListDockName + f" ({self.BoxList.count()})")
        self.labelListDock.setWindowTitle(self.labelListDockName + f" ({self.labelList.count()})")

    def remLabels(self, shapes):
        if shapes is None:
            # print('rm empty label')
            return
        for shape in shapes:
            item = self.shapesToItems[shape]
            self.labelList.takeItem(self.labelList.row(item))
            del self.shapesToItems[shape]
            del self.itemsToShapes[item]
            self.updateComboBox()

            # ADD:
            item = self.shapesToItemsbox[shape]
            self.BoxList.takeItem(self.BoxList.row(item))
            del self.shapesToItemsbox[shape]
            del self.itemsToShapesbox[item]
            self.updateComboBox()
        self.updateIndexList()

    def loadLabels(self, shapes):
        s = []
        shape_index = 0
        for label, points, line_color, key_cls, difficult in shapes:
            shape = Shape(label=label, line_color=line_color, key_cls=key_cls)
            for x, y in points:

                # Ensure the labels are within the bounds of the image. If not, fix them.
                x, y, snapped = self.canvas.snapPointToCanvas(x, y)
                if snapped:
                    self.setDirty()

                shape.addPoint(QPointF(x, y))
            shape.difficult = difficult
            shape.idx = shape_index
            shape_index += 1
            # shape.locked = False
            shape.close()
            s.append(shape)

            self._update_shape_color(shape)
            self.addLabel(shape)

        self.updateComboBox()
        self.canvas.loadShapes(s)

    def singleLabel(self, shape):
        if shape is None:
            # print('rm empty label')
            return
        item = self.shapesToItems[shape]
        item.setText(shape.label)
        self.updateComboBox()

        # ADD:
        item = self.shapesToItemsbox[shape]
        item.setText(str([(int(p.x()), int(p.y())) for p in shape.points]))
        self.updateComboBox()

    def updateComboBox(self):
        # Get the unique labels and add them to the Combobox.
        itemsTextList = [str(self.labelList.item(i).text()) for i in range(self.labelList.count())]

        uniqueTextList = list(set(itemsTextList))
        # Add a null row for showing all the labels
        uniqueTextList.append("")
        uniqueTextList.sort()

        # self.comboBox.update_items(uniqueTextList)

    def updateIndexList(self):
        self.indexList.clear()
        for i in range(self.labelList.count()):
            string = QListWidgetItem(str(i))
            string.setTextAlignment(Qt.AlignHCenter)
            self.indexList.addItem(string)

    def saveLabels(self, annotationFilePath, mode='Auto'):
        # Mode is Auto means that labels will be loaded from self.result_dic totally, which is the output of ocr model
        annotationFilePath = ustr(annotationFilePath)

        def format_shape(s):
            # print('s in saveLabels is ',s)
            return dict(label=s.label,  # str
                        line_color=s.line_color.getRgb(),
                        fill_color=s.fill_color.getRgb(),
                        points=[(int(p.x()), int(p.y())) for p in s.points],  # QPonitF
                        difficult=s.difficult,
                        key_cls=s.key_cls)  # bool

        if mode == 'Auto':
            shapes = []
        else:
            shapes = [format_shape(shape) for shape in self.canvas.shapes if shape.line_color != DEFAULT_LOCK_COLOR]
        # Can add differrent annotation formats here
        for box in self.result_dic:
            trans_dic = {"label": box[1][0], "points": box[0], "difficult": False}
            if self.kie_mode:
                if len(box) == 3:
                    trans_dic.update({"key_cls": box[2]})
                else:
                    trans_dic.update({"key_cls": "None"})
            if trans_dic["label"] == "" and mode == 'Auto':
                continue
            shapes.append(trans_dic)

        try:
            trans_dic = []
            for box in shapes:
                trans_dict = {"transcription": box['label'], "points": box['points'], "difficult": box['difficult']}
                if self.kie_mode:
                    trans_dict.update({"key_cls": box['key_cls']})
                trans_dic.append(trans_dict)
            self.PPlabel[annotationFilePath] = trans_dic
            if mode == 'Auto':
                self.Cachelabel[annotationFilePath] = trans_dic

            # else:
            #     self.labelFile.save(annotationFilePath, shapes, self.filePath, self.imageData,
            #                         self.lineColor.getRgb(), self.fillColor.getRgb())
            # print('Image:{0} -> Annotation:{1}'.format(self.filePath, annotationFilePath))
            return True
        except:
            self.errorMessage(u'Error saving label data', u'Error saving label data')
            return False

    def copySelectedShape(self):
        for shape in self.canvas.copySelectedShape():
            self.addLabel(shape)
        # fix copy and delete
        # self.shapeSelectionChanged(True)

    def move_scrollbar(self, value):
        self.labelListBar.setValue(value)
        self.indexListBar.setValue(value)

    def labelSelectionChanged(self):
        if self._noSelectionSlot:
            return
        if self.canvas.editing():
            selected_shapes = []
            for item in self.labelList.selectedItems():
                selected_shapes.append(self.itemsToShapes[item])
            if selected_shapes:
                self.canvas.selectShapes(selected_shapes)
            else:
                self.canvas.deSelectShape()

    def indexSelectionChanged(self):
        if self._noSelectionSlot:
            return
        if self.canvas.editing():
            selected_shapes = []
            for item in self.indexList.selectedItems():
                # map index item to label item
                index = self.indexList.indexFromItem(item).row()
                item = self.labelList.item(index)
                selected_shapes.append(self.itemsToShapes[item])
            if selected_shapes:
                self.canvas.selectShapes(selected_shapes)
            else:
                self.canvas.deSelectShape()

    def boxSelectionChanged(self):
        if self._noSelectionSlot:
            # self.BoxList.scrollToItem(self.currentBox(), QAbstractItemView.PositionAtCenter)
            return
        if self.canvas.editing():
            selected_shapes = []
            for item in self.BoxList.selectedItems():
                selected_shapes.append(self.itemsToShapesbox[item])
            if selected_shapes:
                self.canvas.selectShapes(selected_shapes)
            else:
                self.canvas.deSelectShape()

    def labelItemChanged(self, item):
        # avoid accidentally triggering the itemChanged siganl with unhashable item
        # Unknown trigger condition
        if type(item) == HashableQListWidgetItem:
            shape = self.itemsToShapes[item]
            label = item.text()
            if label != shape.label:
                shape.label = item.text()
                # shape.line_color = generateColorByText(shape.label)
                self.setDirty()
            elif not ((item.checkState() == Qt.Unchecked) ^ (not shape.difficult)):
                shape.difficult = True if item.checkState() == Qt.Unchecked else False
                self.setDirty()
            else:  # User probably changed item visibility
                self.canvas.setShapeVisible(shape, True)  # item.checkState() == Qt.Checked
                # self.actions.save.setEnabled(True)
        else:
            print('enter labelItemChanged slot with unhashable item: ', item, item.text())
    
    def drag_drop_happened(self):
        '''
        label list drag drop signal slot
        '''
        # print('___________________drag_drop_happened_______________')
        # should only select single item
        for item in self.labelList.selectedItems():
            newIndex = self.labelList.indexFromItem(item).row()

        # only support drag_drop one item
        assert len(self.canvas.selectedShapes) > 0
        for shape in self.canvas.selectedShapes:
            selectedShapeIndex = shape.idx
        
        if newIndex == selectedShapeIndex:
            return

        # move corresponding item in shape list
        shape = self.canvas.shapes.pop(selectedShapeIndex)
        self.canvas.shapes.insert(newIndex, shape)
            
        # update bbox index
        self.canvas.updateShapeIndex()

        # boxList update simultaneously
        item = self.BoxList.takeItem(selectedShapeIndex)
        self.BoxList.insertItem(newIndex, item)

        # changes happen
        self.setDirty()

    # Callback functions:
    def newShape(self, value=True):
        """Pop-up and give focus to the label editor.

        position MUST be in global coordinates.
        """
        if len(self.labelHist) > 0:
            self.labelDialog = LabelDialog(parent=self, listItem=self.labelHist)

        if value:
            text = self.labelDialog.popUp(text=self.prevLabelText)
            self.lastLabel = text
        else:
            text = self.prevLabelText

        if text is not None:
            self.prevLabelText = self.stringBundle.getString('tempLabel')

            shape = self.canvas.setLastLabel(text, None, None, None)  # generate_color, generate_color
            if self.kie_mode:
                key_text, _ = self.keyDialog.popUp(self.key_previous_text)
                if key_text is not None:
                    shape = self.canvas.setLastLabel(text, None, None, key_text)  # generate_color, generate_color
                    self.key_previous_text = key_text
                    if not self.keyList.findItemsByLabel(key_text):
                        item = self.keyList.createItemFromLabel(key_text)
                        self.keyList.addItem(item)
                        rgb = self._get_rgb_by_label(key_text, self.kie_mode)
                        self.keyList.setItemLabel(item, key_text, rgb)

                    self._update_shape_color(shape)
                    self.keyDialog.addLabelHistory(key_text)

            self.addLabel(shape)
            if self.beginner():  # Switch to edit mode.
                self.canvas.setEditing(True)
                self.actions.create.setEnabled(True)
                self.actions.createpoly.setEnabled(True)
                self.actions.undoLastPoint.setEnabled(False)
                self.actions.undo.setEnabled(True)
            else:
                self.actions.editMode.setEnabled(True)
            self.setDirty()

        else:
            # self.canvas.undoLastLine()
            self.canvas.resetAllLines()

    def _update_shape_color(self, shape):
        r, g, b = self._get_rgb_by_label(shape.key_cls, self.kie_mode)
        shape.line_color = QColor(r, g, b)
        shape.vertex_fill_color = QColor(r, g, b)
        shape.hvertex_fill_color = QColor(255, 255, 255)
        shape.fill_color = QColor(r, g, b, 128)
        shape.select_line_color = QColor(255, 255, 255)
        shape.select_fill_color = QColor(r, g, b, 155)

    def _get_rgb_by_label(self, label, kie_mode):
        shift_auto_shape_color = 2  # use for random color
        if kie_mode and label != "None":
            item = self.keyList.findItemsByLabel(label)[0]
            label_id = self.keyList.indexFromItem(item).row() + 1
            label_id += shift_auto_shape_color
            return LABEL_COLORMAP[label_id % len(LABEL_COLORMAP)]
        else:
            return (0, 255, 0)

    def scrollRequest(self, delta, orientation):
        units = - delta / (8 * 15)
        bar = self.scrollBars[orientation]
        bar.setValue(bar.value() + bar.singleStep() * units)

    def setZoom(self, value):
        self.actions.fitWidth.setChecked(False)
        self.actions.fitWindow.setChecked(False)
        self.zoomMode = self.MANUAL_ZOOM
        self.zoomWidget.setValue(value)

    def addZoom(self, increment=10):
        self.setZoom(self.zoomWidget.value() + increment)
        self.imageSlider.setValue(self.zoomWidget.value() + increment)  # set zoom slider value

    def zoomRequest(self, delta):
        # get the current scrollbar positions
        # calculate the percentages ~ coordinates
        h_bar = self.scrollBars[Qt.Horizontal]
        v_bar = self.scrollBars[Qt.Vertical]

        # get the current maximum, to know the difference after zooming
        h_bar_max = h_bar.maximum()
        v_bar_max = v_bar.maximum()

        # get the cursor position and canvas size
        # calculate the desired movement from 0 to 1
        # where 0 = move left
        #       1 = move right
        # up and down analogous
        cursor = QCursor()
        pos = cursor.pos()
        relative_pos = QWidget.mapFromGlobal(self, pos)

        cursor_x = relative_pos.x()
        cursor_y = relative_pos.y()

        w = self.scrollArea.width()
        h = self.scrollArea.height()

        # the scaling from 0 to 1 has some padding
        # you don't have to hit the very leftmost pixel for a maximum-left movement
        margin = 0.1
        move_x = (cursor_x - margin * w) / (w - 2 * margin * w)
        move_y = (cursor_y - margin * h) / (h - 2 * margin * h)

        # clamp the values from 0 to 1
        move_x = min(max(move_x, 0), 1)
        move_y = min(max(move_y, 0), 1)

        # zoom in
        units = delta / (8 * 15)
        scale = 10
        self.addZoom(scale * units)

        # get the difference in scrollbar values
        # this is how far we can move
        d_h_bar_max = h_bar.maximum() - h_bar_max
        d_v_bar_max = v_bar.maximum() - v_bar_max

        # get the new scrollbar values
        new_h_bar_value = h_bar.value() + move_x * d_h_bar_max
        new_v_bar_value = v_bar.value() + move_y * d_v_bar_max

        h_bar.setValue(new_h_bar_value)
        v_bar.setValue(new_v_bar_value)

    def setFitWindow(self, value=True):
        if value:
            self.actions.fitWidth.setChecked(False)
        self.zoomMode = self.FIT_WINDOW if value else self.MANUAL_ZOOM
        self.adjustScale()

    def setFitWidth(self, value=True):
        if value:
            self.actions.fitWindow.setChecked(False)
        self.zoomMode = self.FIT_WIDTH if value else self.MANUAL_ZOOM
        self.adjustScale()

    def togglePolygons(self, value):
        for item, shape in self.itemsToShapes.items():
            self.canvas.setShapeVisible(shape, value)

    def loadFile(self, filePath=None):
        """Load the specified file, or the last opened file if None."""
        if self.dirty:
            self.mayContinue()
        self.resetState()
        self.canvas.setEnabled(False)
        if filePath is None:
            filePath = self.settings.get(SETTING_FILENAME)

        # Make sure that filePath is a regular python string, rather than QString
        filePath = ustr(filePath)
        # Fix bug: An index error after select a directory when open a new file.
        unicodeFilePath = ustr(filePath)
        # unicodeFilePath = os.path.abspath(unicodeFilePath)
        # Tzutalin 20160906 : Add file list and dock to move faster
        # Highlight the file item

        if unicodeFilePath and self.fileListWidget.count() > 0:
            if unicodeFilePath in self.mImgList:
                index = self.mImgList.index(unicodeFilePath)
                fileWidgetItem = self.fileListWidget.item(index)
                print('unicodeFilePath is', unicodeFilePath)
                fileWidgetItem.setSelected(True)
                self.iconlist.clear()
                self.additems5(None)

                for i in range(5):
                    item_tooltip = self.iconlist.item(i).toolTip()
                    # print(i,"---",item_tooltip)
                    if item_tooltip == ustr(filePath):
                        titem = self.iconlist.item(i)
                        titem.setSelected(True)
                        self.iconlist.scrollToItem(titem)
                        break
            else:
                self.fileListWidget.clear()
                self.mImgList.clear()
                self.iconlist.clear()

        # if unicodeFilePath and self.iconList.count() > 0:
        #     if unicodeFilePath in self.mImgList:

        if unicodeFilePath and os.path.exists(unicodeFilePath):
            self.canvas.verified = False
            cvimg = cv2.imdecode(np.fromfile(unicodeFilePath, dtype=np.uint8), 1)
            height, width, depth = cvimg.shape
            cvimg = cv2.cvtColor(cvimg, cv2.COLOR_BGR2RGB)
            image = QImage(cvimg.data, width, height, width * depth, QImage.Format_RGB888)

            if image.isNull():
                self.errorMessage(u'Error opening file',
                                  u"<p>Make sure <i>%s</i> is a valid image file." % unicodeFilePath)
                self.status("Error reading %s" % unicodeFilePath)
                return False
            self.status("Loaded %s" % os.path.basename(unicodeFilePath))
            self.image = image
            self.filePath = unicodeFilePath
            self.canvas.loadPixmap(QPixmap.fromImage(image))

            if self.validFilestate(filePath) is True:
                self.setClean()
            else:
                self.dirty = False
                self.actions.save.setEnabled(True)
            if len(self.canvas.lockedShapes) != 0:
                self.actions.save.setEnabled(True)
                self.setDirty()
            self.canvas.setEnabled(True)
            self.adjustScale(initial=True)
            self.paintCanvas()
            self.addRecentFile(self.filePath)
            self.toggleActions(True)

            self.showBoundingBoxFromPPlabel(filePath)

            self.setWindowTitle(__appname__ + ' ' + filePath)

            # Default : select last item if there is at least one item
            if self.labelList.count():
                self.labelList.setCurrentItem(self.labelList.item(self.labelList.count() - 1))
                self.labelList.item(self.labelList.count() - 1).setSelected(True)
                self.indexList.item(self.labelList.count() - 1).setSelected(True)

            # show file list image count
            select_indexes = self.fileListWidget.selectedIndexes()
            if len(select_indexes) > 0:
                self.fileDock.setWindowTitle(self.fileListName + f" ({select_indexes[0].row() + 1}"
                                                                 f"/{self.fileListWidget.count()})")
            # update show counting
            self.BoxListDock.setWindowTitle(self.BoxListDockName + f" ({self.BoxList.count()})")
            self.labelListDock.setWindowTitle(self.labelListDockName + f" ({self.labelList.count()})")

            self.canvas.setFocus(True)
            return True
        return False

    def showBoundingBoxFromPPlabel(self, filePath):
        width, height = self.image.width(), self.image.height()
        imgidx = self.getImglabelidx(filePath)
        shapes = []
        # box['ratio'] of the shapes saved in lockedShapes contains the ratio of the
        # four corner coordinates of the shapes to the height and width of the image
        for box in self.canvas.lockedShapes:
            key_cls = 'None' if not self.kie_mode else box['key_cls']
            if self.canvas.isInTheSameImage:
                shapes.append((box['transcription'], [[s[0] * width, s[1] * height] for s in box['ratio']],
                               DEFAULT_LOCK_COLOR, key_cls, box['difficult']))
            else:
                shapes.append(('锁定框：待检测', [[s[0] * width, s[1] * height] for s in box['ratio']],
                               DEFAULT_LOCK_COLOR, key_cls, box['difficult']))
        if imgidx in self.PPlabel.keys():
            for box in self.PPlabel[imgidx]:
                key_cls = 'None' if not self.kie_mode else box.get('key_cls', 'None')
                shapes.append((box['transcription'], box['points'], None, key_cls, box.get('difficult', False)))

        if shapes != []:
            self.loadLabels(shapes)
            self.canvas.verified = False

    def validFilestate(self, filePath):
        if filePath not in self.fileStatedict.keys():
            return None
        elif self.fileStatedict[filePath] == 1:
            return True
        else:
            return False

    def resizeEvent(self, event):
        if self.canvas and not self.image.isNull() \
                and self.zoomMode != self.MANUAL_ZOOM:
            self.adjustScale()
        super(MainWindow, self).resizeEvent(event)

    def paintCanvas(self):
        assert not self.image.isNull(), "cannot paint null image"
        self.canvas.scale = 0.01 * self.zoomWidget.value()
        self.canvas.adjustSize()
        self.canvas.update()

    def adjustScale(self, initial=False):
        value = self.scalers[self.FIT_WINDOW if initial else self.zoomMode]()
        self.zoomWidget.setValue(int(100 * value))
        self.imageSlider.setValue(self.zoomWidget.value())  # set zoom slider value

    def scaleFitWindow(self):
        """Figure out the size of the pixmap in order to fit the main widget."""
        e = 2.0  # So that no scrollbars are generated.
        w1 = self.centralWidget().width() - e
        h1 = self.centralWidget().height() - e - 110
        a1 = w1 / h1
        # Calculate a new scale value based on the pixmap's aspect ratio.
        w2 = self.canvas.pixmap.width() - 0.0
        h2 = self.canvas.pixmap.height() - 0.0
        a2 = w2 / h2
        return w1 / w2 if a2 >= a1 else h1 / h2

    def scaleFitWidth(self):
        # The epsilon does not seem to work too well here.
        w = self.centralWidget().width() - 2.0
        return w / self.canvas.pixmap.width()

    def closeEvent(self, event):
        if not self.mayContinue():
            event.ignore()
        else:
            settings = self.settings
            # If it loads images from dir, don't load it at the beginning
            if self.dirname is None:
                settings[SETTING_FILENAME] = self.filePath if self.filePath else ''
            else:
                settings[SETTING_FILENAME] = ''

            settings[SETTING_WIN_SIZE] = self.size()
            settings[SETTING_WIN_POSE] = self.pos()
            settings[SETTING_WIN_STATE] = self.saveState()
            settings[SETTING_LINE_COLOR] = self.lineColor
            settings[SETTING_FILL_COLOR] = self.fillColor
            settings[SETTING_RECENT_FILES] = self.recentFiles
            settings[SETTING_ADVANCE_MODE] = not self._beginner
            if self.defaultSaveDir and os.path.exists(self.defaultSaveDir):
                settings[SETTING_SAVE_DIR] = ustr(self.defaultSaveDir)
            else:
                settings[SETTING_SAVE_DIR] = ''

            if self.lastOpenDir and os.path.exists(self.lastOpenDir):
                settings[SETTING_LAST_OPEN_DIR] = self.lastOpenDir
            else:
                settings[SETTING_LAST_OPEN_DIR] = ''

            settings[SETTING_PAINT_LABEL] = self.displayLabelOption.isChecked()
            settings[SETTING_PAINT_INDEX] = self.displayIndexOption.isChecked()
            settings[SETTING_DRAW_SQUARE] = self.drawSquaresOption.isChecked()
            settings.save()
            try:
                self.saveLabelFile()
            except:
                pass

    def loadRecent(self, filename):
        if self.mayContinue():
            print(filename, "======")
            self.loadFile(filename)

    def scanAllImages(self, folderPath):
        extensions = ['.%s' % fmt.data().decode("ascii").lower() for fmt in QImageReader.supportedImageFormats()]
        images = []

        for file in os.listdir(folderPath):
            if file.lower().endswith(tuple(extensions)):
                relativePath = os.path.join(folderPath, file)
                path = ustr(os.path.abspath(relativePath))
                images.append(path)
        natural_sort(images, key=lambda x: x.lower())
        return images

    def openDirDialog(self, _value=False, dirpath=None, silent=False):
        if not self.mayContinue():
            return

        defaultOpenDirPath = dirpath if dirpath else '.'
        if self.lastOpenDir and os.path.exists(self.lastOpenDir):
            defaultOpenDirPath = self.lastOpenDir
        else:
            defaultOpenDirPath = os.path.dirname(self.filePath) if self.filePath else '.'
        if silent != True:
            targetDirPath = ustr(QFileDialog.getExistingDirectory(self,
                                                                  '%s - Open Directory' % __appname__,
                                                                  defaultOpenDirPath,
                                                                  QFileDialog.ShowDirsOnly | QFileDialog.DontResolveSymlinks))
        else:
            targetDirPath = ustr(defaultOpenDirPath)
        self.lastOpenDir = targetDirPath
        self.importDirImages(targetDirPath)

    def openDatasetDirDialog(self):
        if self.lastOpenDir and os.path.exists(self.lastOpenDir):
            if platform.system() == 'Windows':
                os.startfile(self.lastOpenDir)
            else:
                os.system('open ' + os.path.normpath(self.lastOpenDir))
            defaultOpenDirPath = self.lastOpenDir

        else:
            if self.lang == 'ch':
                self.msgBox.warning(self, "提示", "\n 原文件夹已不存在,请从新选择数据集路径!")
            else:
                self.msgBox.warning(self, "Warn",
                                    "\n The original folder no longer exists, please choose the data set path again!")

            self.actions.open_dataset_dir.setEnabled(False)
            defaultOpenDirPath = os.path.dirname(self.filePath) if self.filePath else '.'

    def init_key_list(self, label_dict):
        if not self.kie_mode:
            return
        # load key_cls
        for image, info in label_dict.items():
            for box in info:
                if "key_cls" not in box:
                    box.update({"key_cls": "None"})
                self.existed_key_cls_set.add(box["key_cls"])
        if len(self.existed_key_cls_set) > 0:
            for key_text in self.existed_key_cls_set:
                if not self.keyList.findItemsByLabel(key_text):
                    item = self.keyList.createItemFromLabel(key_text)
                    self.keyList.addItem(item)
                    rgb = self._get_rgb_by_label(key_text, self.kie_mode)
                    self.keyList.setItemLabel(item, key_text, rgb)

        if self.keyDialog is None:
            # key list dialog
            self.keyDialog = KeyDialog(
                text=self.key_dialog_tip,
                parent=self,
                labels=self.existed_key_cls_set,
                sort_labels=True,
                show_text_field=True,
                completion="startswith",
                fit_to_content={'column': True, 'row': False},
                flags=None
            )

    def importDirImages(self, dirpath, isDelete=False):
        if not self.mayContinue() or not dirpath:
            return
        if self.defaultSaveDir and self.defaultSaveDir != dirpath:
            self.saveLabelFile()

        if not isDelete:
            self.loadFilestate(dirpath)
            self.PPlabelpath = dirpath + '/Label.txt'
            self.PPlabel = self.loadLabelFile(self.PPlabelpath)
            self.Cachelabelpath = dirpath + '/Cache.cach'
            self.Cachelabel = self.loadLabelFile(self.Cachelabelpath)
            if self.Cachelabel:
                self.PPlabel = dict(self.Cachelabel, **self.PPlabel)

            self.init_key_list(self.PPlabel)

        self.lastOpenDir = dirpath
        self.dirname = dirpath

        self.defaultSaveDir = dirpath
        self.statusBar().showMessage('%s started. Annotation will be saved to %s' %
                                     (__appname__, self.defaultSaveDir))
        self.statusBar().show()

        self.filePath = None
        self.fileListWidget.clear()
        self.mImgList = self.scanAllImages(dirpath)
        self.mImgList5 = self.mImgList[:5]
        self.openNextImg()
        doneicon = newIcon('done')
        closeicon = newIcon('close')
        for imgPath in self.mImgList:
            filename = os.path.basename(imgPath)
            if self.validFilestate(imgPath) is True:
                item = QListWidgetItem(doneicon, filename)
            else:
                item = QListWidgetItem(closeicon, filename)
            self.fileListWidget.addItem(item)

        print('DirPath in importDirImages is', dirpath)
        self.iconlist.clear()
        self.additems5(dirpath)
        self.changeFileFolder = True
        self.haveAutoReced = False
        self.AutoRecognition.setEnabled(True)
        self.reRecogButton.setEnabled(True)
        self.tableRecButton.setEnabled(True)
        self.actions.AutoRec.setEnabled(True)
        self.actions.reRec.setEnabled(True)
        self.actions.tableRec.setEnabled(True)
        self.actions.open_dataset_dir.setEnabled(True)
        self.actions.rotateLeft.setEnabled(True)
        self.actions.rotateRight.setEnabled(True)

        self.fileListWidget.setCurrentRow(0)  # set list index to first
        self.fileDock.setWindowTitle(self.fileListName + f" (1/{self.fileListWidget.count()})")  # show image count

    def openPrevImg(self, _value=False):
        if len(self.mImgList) <= 0:
            return

        if self.filePath is None:
            return

        currIndex = self.mImgList.index(self.filePath)
        self.mImgList5 = self.mImgList[:5]
        if currIndex - 1 >= 0:
            filename = self.mImgList[currIndex - 1]
            self.mImgList5 = self.indexTo5Files(currIndex - 1)
            if filename:
                self.loadFile(filename)

    def openNextImg(self, _value=False):
        if not self.mayContinue():
            return

        if len(self.mImgList) <= 0:
            return

        filename = None
        if self.filePath is None:
            filename = self.mImgList[0]
            self.mImgList5 = self.mImgList[:5]
        else:
            currIndex = self.mImgList.index(self.filePath)
            if currIndex + 1 < len(self.mImgList):
                filename = self.mImgList[currIndex + 1]
                self.mImgList5 = self.indexTo5Files(currIndex + 1)
            else:
                self.mImgList5 = self.indexTo5Files(currIndex)
        if filename:
            print('file name in openNext is ', filename)
            self.loadFile(filename)

    def updateFileListIcon(self, filename):
        pass

    def saveFile(self, _value=False, mode='Manual'):
        # Manual mode is used for users click "Save" manually,which will change the state of the image
        if self.filePath:
            imgidx = self.getImglabelidx(self.filePath)
            self._saveFile(imgidx, mode=mode)

    def saveLockedShapes(self):
        self.canvas.lockedShapes = []
        self.canvas.selectedShapes = []
        for s in self.canvas.shapes:
            if s.line_color == DEFAULT_LOCK_COLOR:
                self.canvas.selectedShapes.append(s)
        self.lockSelectedShape()
        for s in self.canvas.shapes:
            if s.line_color == DEFAULT_LOCK_COLOR:
                self.canvas.selectedShapes.remove(s)
                self.canvas.shapes.remove(s)

    def _saveFile(self, annotationFilePath, mode='Manual'):
        if len(self.canvas.lockedShapes) != 0:
            self.saveLockedShapes()

        if mode == 'Manual':
            self.result_dic_locked = []
            img = cv2.imread(self.filePath)
            width, height = self.image.width(), self.image.height()
            for shape in self.canvas.lockedShapes:
                box = [[int(p[0] * width), int(p[1] * height)] for p in shape['ratio']]
                # assert len(box) == 4
                result = [(shape['transcription'], 1)]
                result.insert(0, box)
                self.result_dic_locked.append(result)
            self.result_dic += self.result_dic_locked
            self.result_dic_locked = []
            if annotationFilePath and self.saveLabels(annotationFilePath, mode=mode):
                self.setClean()
                self.statusBar().showMessage('Saved to  %s' % annotationFilePath)
                self.statusBar().show()
                currIndex = self.mImgList.index(self.filePath)
                item = self.fileListWidget.item(currIndex)
                item.setIcon(newIcon('done'))

                self.fileStatedict[self.filePath] = 1
                if len(self.fileStatedict) % self.autoSaveNum == 0:
                    self.saveFilestate()
                    self.savePPlabel(mode='Auto')

                self.fileListWidget.insertItem(int(currIndex), item)
                if not self.canvas.isInTheSameImage:
                    self.openNextImg()
                self.actions.saveRec.setEnabled(True)
                self.actions.saveLabel.setEnabled(True)
                self.actions.exportJSON.setEnabled(True) 

        elif mode == 'Auto':
            if annotationFilePath and self.saveLabels(annotationFilePath, mode=mode):
                self.setClean()
                self.statusBar().showMessage('Saved to  %s' % annotationFilePath)
                self.statusBar().show()

    def closeFile(self, _value=False):
        if not self.mayContinue():
            return
        self.resetState()
        self.setClean()
        self.toggleActions(False)
        self.canvas.setEnabled(False)
        self.actions.saveAs.setEnabled(False)

    def deleteImg(self):
        deletePath = self.filePath
        if deletePath is not None:
            deleteInfo = self.deleteImgDialog()
            if deleteInfo == QMessageBox.Yes:
                if platform.system() == 'Windows':
                    from win32com.shell import shell, shellcon
                    shell.SHFileOperation((0, shellcon.FO_DELETE, deletePath, None,
                                           shellcon.FOF_SILENT | shellcon.FOF_ALLOWUNDO | shellcon.FOF_NOCONFIRMATION,
                                           None, None))
                    # linux
                elif platform.system() == 'Linux':
                    cmd = 'trash ' + deletePath
                    os.system(cmd)
                    # macOS
                elif platform.system() == 'Darwin':
                    import subprocess
                    absPath = os.path.abspath(deletePath).replace('\\', '\\\\').replace('"', '\\"')
                    cmd = ['osascript', '-e',
                           'tell app "Finder" to move {the POSIX file "' + absPath + '"} to trash']
                    print(cmd)
                    subprocess.call(cmd, stdout=open(os.devnull, 'w'))

                if self.filePath in self.fileStatedict.keys():
                    self.fileStatedict.pop(self.filePath)
                imgidx = self.getImglabelidx(self.filePath)
                if imgidx in self.PPlabel.keys():
                    self.PPlabel.pop(imgidx)
                self.openNextImg()
                self.importDirImages(self.lastOpenDir, isDelete=True)

    def deleteImgDialog(self):
        yes, cancel = QMessageBox.Yes, QMessageBox.Cancel
        msg = u'The image will be deleted to the recycle bin'
        return QMessageBox.warning(self, u'Attention', msg, yes | cancel)

    def resetAll(self):
        self.settings.reset()
        self.close()
        proc = QProcess()
        proc.startDetached(os.path.abspath(__file__))

    def mayContinue(self):  #
        if not self.dirty:
            return True
        else:
            discardChanges = self.discardChangesDialog()
            if discardChanges == QMessageBox.No:
                return True
            elif discardChanges == QMessageBox.Yes:
                self.canvas.isInTheSameImage = True
                self.saveFile()
                self.canvas.isInTheSameImage = False
                return True
            else:
                return False

    def discardChangesDialog(self):
        yes, no, cancel = QMessageBox.Yes, QMessageBox.No, QMessageBox.Cancel
        if self.lang == 'ch':
            msg = u'您有未保存的变更, 您想保存再继续吗?\n点击 "No" 丢弃所有未保存的变更.'
        else:
            msg = u'You have unsaved changes, would you like to save them and proceed?\nClick "No" to undo all changes.'
        return QMessageBox.warning(self, u'Attention', msg, yes | no | cancel)

    def errorMessage(self, title, message):
        return QMessageBox.critical(self, title,
                                    '<p><b>%s</b></p>%s' % (title, message))

    def currentPath(self):
        return os.path.dirname(self.filePath) if self.filePath else '.'

    def chooseColor(self):
        color = self.colorDialog.getColor(self.lineColor, u'Choose line color',
                                          default=DEFAULT_LINE_COLOR)
        if color:
            self.lineColor = color
            Shape.line_color = color
            self.canvas.setDrawingColor(color)
            self.canvas.update()
            self.setDirty()

    def deleteSelectedShape(self):
        self.remLabels(self.canvas.deleteSelected())
        self.actions.undo.setEnabled(True)
        self.setDirty()
        if self.noShapes():
            for action in self.actions.onShapesPresent:
                action.setEnabled(False)
        self.BoxListDock.setWindowTitle(self.BoxListDockName + f" ({self.BoxList.count()})")
        self.labelListDock.setWindowTitle(self.labelListDockName + f" ({self.labelList.count()})")

    def chshapeLineColor(self):
        color = self.colorDialog.getColor(self.lineColor, u'Choose line color',
                                          default=DEFAULT_LINE_COLOR)
        if color:
            for shape in self.canvas.selectedShapes: shape.line_color = color
            self.canvas.update()
            self.setDirty()

    def chshapeFillColor(self):
        color = self.colorDialog.getColor(self.fillColor, u'Choose fill color',
                                          default=DEFAULT_FILL_COLOR)
        if color:
            for shape in self.canvas.selectedShapes: shape.fill_color = color
            self.canvas.update()
            self.setDirty()

    def copyShape(self):
        self.canvas.endMove(copy=True)
        self.addLabel(self.canvas.selectedShape)
        self.setDirty()

    def moveShape(self):
        self.canvas.endMove(copy=False)
        self.setDirty()

    def loadPredefinedClasses(self, predefClassesFile):
        if os.path.exists(predefClassesFile) is True:
            with codecs.open(predefClassesFile, 'r', 'utf8') as f:
                for line in f:
                    line = line.strip()
                    if self.labelHist is None:
                        self.labelHist = [line]
                    else:
                        self.labelHist.append(line)

    def togglePaintLabelsOption(self):
        self.displayIndexOption.setChecked(False)
        for shape in self.canvas.shapes:
            shape.paintLabel = self.displayLabelOption.isChecked()
            shape.paintIdx = self.displayIndexOption.isChecked()
        self.canvas.repaint()

    def togglePaintIndexOption(self):
        self.displayLabelOption.setChecked(False)
        for shape in self.canvas.shapes:
            shape.paintLabel = self.displayLabelOption.isChecked()
            shape.paintIdx = self.displayIndexOption.isChecked()
        self.canvas.repaint()

    def toogleDrawSquare(self):
        self.canvas.setDrawingShapeToSquare(self.drawSquaresOption.isChecked())

    def additems(self, dirpath):
        for file in self.mImgList:
            pix = QPixmap(file)
            _, filename = os.path.split(file)
            filename, _ = os.path.splitext(filename)
            item = QListWidgetItem(QIcon(pix.scaled(100, 100, Qt.IgnoreAspectRatio, Qt.FastTransformation)),
                                   filename[:10])
            item.setToolTip(file)
            self.iconlist.addItem(item)

    def additems5(self, dirpath):
        for file in self.mImgList5:
            pix = QPixmap(file)
            _, filename = os.path.split(file)
            filename, _ = os.path.splitext(filename)
            pfilename = filename[:10]
            if len(pfilename) < 10:
                lentoken = 12 - len(pfilename)
                prelen = lentoken // 2
                bfilename = prelen * " " + pfilename + (lentoken - prelen) * " "
            # item = QListWidgetItem(QIcon(pix.scaled(100, 100, Qt.KeepAspectRatio, Qt.SmoothTransformation)),filename[:10])
            item = QListWidgetItem(QIcon(pix.scaled(100, 100, Qt.IgnoreAspectRatio, Qt.FastTransformation)), pfilename)
            # item.setForeground(QBrush(Qt.white))
            item.setToolTip(file)
            self.iconlist.addItem(item)
        owidth = 0
        for index in range(len(self.mImgList5)):
            item = self.iconlist.item(index)
            itemwidget = self.iconlist.visualItemRect(item)
            owidth += itemwidget.width()
        self.iconlist.setMinimumWidth(owidth + 50)

    def gen_quad_from_poly(self, poly):
        """
        Generate min area quad from poly.
        """
        point_num = poly.shape[0]
        min_area_quad = np.zeros((4, 2), dtype=np.float32)
        rect = cv2.minAreaRect(poly.astype(
            np.int32))  # (center (x,y), (width, height), angle of rotation)
        box = np.array(cv2.boxPoints(rect))

        first_point_idx = 0
        min_dist = 1e4
        for i in range(4):
            dist = np.linalg.norm(box[(i + 0) % 4] - poly[0]) + \
                   np.linalg.norm(box[(i + 1) % 4] - poly[point_num // 2 - 1]) + \
                   np.linalg.norm(box[(i + 2) % 4] - poly[point_num // 2]) + \
                   np.linalg.norm(box[(i + 3) % 4] - poly[-1])
            if dist < min_dist:
                min_dist = dist
                first_point_idx = i
        for i in range(4):
            min_area_quad[i] = box[(first_point_idx + i) % 4]

        bbox_new = min_area_quad.tolist()
        bbox = []

        for box in bbox_new:
            box = list(map(int, box))
            bbox.append(box)

        return bbox

    def getImglabelidx(self, filePath):
        if platform.system() == 'Windows':
            spliter = '\\'
        else:
            spliter = '/'
        filepathsplit = filePath.split(spliter)[-2:]
        return filepathsplit[0] + '/' + filepathsplit[1]

    def autoRecognition(self):
        assert self.mImgList is not None
        print('Using model from ', self.model)

        uncheckedList = [i for i in self.mImgList if i not in self.fileStatedict.keys()]
        self.autoDialog = AutoDialog(parent=self, ocr=self.ocr, mImgList=uncheckedList, lenbar=len(uncheckedList))
        self.autoDialog.popUp()
        self.currIndex = len(self.mImgList) - 1
        self.loadFile(self.filePath)  # ADD
        self.haveAutoReced = True
        self.AutoRecognition.setEnabled(False)
        self.actions.AutoRec.setEnabled(False)
        self.setDirty()
        self.saveCacheLabel()

        self.init_key_list(self.Cachelabel)

    def reRecognition(self):
        img = cv2.imdecode(np.fromfile(self.filePath,dtype=np.uint8),1)
        # org_box = [dic['points'] for dic in self.PPlabel[self.getImglabelidx(self.filePath)]]
        if self.canvas.shapes:
            self.result_dic = []
            self.result_dic_locked = []  # result_dic_locked stores the ocr result of self.canvas.lockedShapes
            rec_flag = 0
            for shape in self.canvas.shapes:
                box = [[int(p.x()), int(p.y())] for p in shape.points]
                kie_cls = shape.key_cls

                if len(box) > 4:
                    box = self.gen_quad_from_poly(np.array(box))
                assert len(box) == 4

                img_crop = get_rotate_crop_image(img, np.array(box, np.float32))
                if img_crop is None:
                    msg = 'Can not recognise the detection box in ' + self.filePath + '. Please change manually'
                    QMessageBox.information(self, "Information", msg)
                    return
                result = self.ocr.ocr(img_crop, cls=True, det=False)[0]
                if result[0][0] != '':
                    if shape.line_color == DEFAULT_LOCK_COLOR:
                        shape.label = result[0][0]
                        result.insert(0, box)
                        if self.kie_mode:
                            result.append(kie_cls)
                        self.result_dic_locked.append(result)
                    else:
                        result.insert(0, box)
                        if self.kie_mode:
                            result.append(kie_cls)
                        self.result_dic.append(result)
                else:
                    print('Can not recognise the box')
                    if shape.line_color == DEFAULT_LOCK_COLOR:
                        shape.label = result[0][0]
                        if self.kie_mode:
                            self.result_dic_locked.append([box, (self.noLabelText, 0), kie_cls])
                        else:
                            self.result_dic_locked.append([box, (self.noLabelText, 0)])
                    else:
                        if self.kie_mode:
                            self.result_dic.append([box, (self.noLabelText, 0), kie_cls])
                        else:
                            self.result_dic.append([box, (self.noLabelText, 0)])
                try:
                    if self.noLabelText == shape.label or result[1][0] == shape.label:
                        print('label no change')
                    else:
                        rec_flag += 1
                except IndexError as e:
                    print('Can not recognise the box')
            if (len(self.result_dic) > 0 and rec_flag > 0) or self.canvas.lockedShapes:
                self.canvas.isInTheSameImage = True
                self.saveFile(mode='Auto')
                self.loadFile(self.filePath)
                self.canvas.isInTheSameImage = False
                self.setDirty()
            elif len(self.result_dic) == len(self.canvas.shapes) and rec_flag == 0:
                if self.lang == 'ch':
                    QMessageBox.information(self, "Information", "识别结果保持一致！")
                else:
                    QMessageBox.information(self, "Information", "The recognition result remains unchanged!")
            else:
                print('Can not recgonise in ', self.filePath)
        else:
            QMessageBox.information(self, "Information", "Draw a box!")

    def singleRerecognition(self):
        img = cv2.imdecode(np.fromfile(self.filePath,dtype=np.uint8),1)
        for shape in self.canvas.selectedShapes:
            box = [[int(p.x()), int(p.y())] for p in shape.points]
            if len(box) > 4:
                box = self.gen_quad_from_poly(np.array(box))
            assert len(box) == 4
            img_crop = get_rotate_crop_image(img, np.array(box, np.float32))
            if img_crop is None:
                msg = 'Can not recognise the detection box in ' + self.filePath + '. Please change manually'
                QMessageBox.information(self, "Information", msg)
                return
            result = self.ocr.ocr(img_crop, cls=True, det=False)[0]
            if result[0][0] != '':
                result.insert(0, box)
                print('result in reRec is ', result)
                if result[1][0] == shape.label:
                    print('label no change')
                else:
                    shape.label = result[1][0]
            else:
                print('Can not recognise the box')
                if self.noLabelText == shape.label:
                    print('label no change')
                else:
                    shape.label = self.noLabelText
            self.singleLabel(shape)
            self.setDirty()

    def TableRecognition(self):
        '''
            Table Recegnition
        '''
        from paddleocr import to_excel

        import time

        start = time.time()
        img = cv2.imread(self.filePath)
        res = self.table_ocr(img, return_ocr_result_in_table=True)

        TableRec_excel_dir = self.lastOpenDir + '/tableRec_excel_output/'
        os.makedirs(TableRec_excel_dir, exist_ok=True)
        filename, _ = os.path.splitext(os.path.basename(self.filePath))

        excel_path = TableRec_excel_dir + '{}.xlsx'.format(filename)
        
        if res is None:
            msg = 'Can not recognise the table in ' + self.filePath + '. Please change manually'
            QMessageBox.information(self, "Information", msg)
            to_excel('', excel_path) # create an empty excel
            return
        
        # save res
        # ONLY SUPPORT ONE TABLE in one image
        hasTable = False
        for region in res:
            if region['type'] == 'table':
                if region['res']['boxes'] is None:
                    msg = 'Can not recognise the detection box in ' + self.filePath + '. Please change manually'
                    QMessageBox.information(self, "Information", msg)
                    to_excel('', excel_path) # create an empty excel
                    return
                hasTable = True
                # save table ocr result on PPOCRLabel
                # clear all old annotaions before saving result
                self.itemsToShapes.clear()
                self.shapesToItems.clear()
                self.itemsToShapesbox.clear()  # ADD
                self.shapesToItemsbox.clear()
                self.labelList.clear()
                self.indexList.clear()
                self.BoxList.clear()
                self.result_dic = []
                self.result_dic_locked = []

                shapes = []
                result_len = len(region['res']['boxes'])
                order_index = 0
                for i in range(result_len):
                    bbox = np.array(region['res']['boxes'][i])
                    rec_text = region['res']['rec_res'][i][0]

                    rext_bbox = [[bbox[0], bbox[1]], [bbox[2], bbox[1]], [bbox[2], bbox[3]], [bbox[0], bbox[3]]]

                    # save bbox to shape
                    shape = Shape(label=rec_text, line_color=DEFAULT_LINE_COLOR, key_cls=None)
                    for point in rext_bbox:
                        x, y = point
                        # Ensure the labels are within the bounds of the image. 
                        # If not, fix them.
                        x, y, snapped = self.canvas.snapPointToCanvas(x, y)
                        shape.addPoint(QPointF(x, y))
                    shape.difficult = False
                    shape.idx = order_index
                    order_index += 1
                    # shape.locked = False
                    shape.close()
                    self.addLabel(shape)
                    shapes.append(shape)
                self.setDirty()
                self.canvas.loadShapes(shapes)
                
                # save HTML result to excel
                try:
                    to_excel(region['res']['html'], excel_path)
                except:
                    print('Can not save excel file, maybe Permission denied (.xlsx is being occupied)')
                break
        
        if not hasTable:
            msg = 'Can not recognise the table in ' + self.filePath + '. Please change manually'
            QMessageBox.information(self, "Information", msg)
            to_excel('', excel_path) # create an empty excel
            return

        # automatically open excel annotation file
        if platform.system() == 'Windows':
            try:
                import win32com.client
            except:
                print("CANNOT OPEN .xlsx. It could be one of the following reasons: " \
                    "Only support Windows | No python win32com")

            try:
                xl = win32com.client.Dispatch("Excel.Application")
                xl.Visible = True
                xl.Workbooks.Open(excel_path)
                # excelEx = "You need to show the excel executable at this point"
                # subprocess.Popen([excelEx, excel_path])

                # os.startfile(excel_path)
            except:
                print("CANNOT OPEN .xlsx. It could be the following reasons: " \
                    ".xlsx is not existed")
        else:
            os.system('open ' + os.path.normpath(excel_path))
                
        print('time cost: ', time.time() - start)

    def cellreRecognition(self):
        '''
            re-recognise text in a cell
        '''
        img = cv2.imread(self.filePath)
        for shape in self.canvas.selectedShapes:
            box = [[int(p.x()), int(p.y())] for p in shape.points]

            if len(box) > 4:
                box = self.gen_quad_from_poly(np.array(box))
            assert len(box) == 4

            # pad around bbox for better text recognition accuracy
            _box = boxPad(box, img.shape, 6)
            img_crop = get_rotate_crop_image(img, np.array(_box, np.float32))
            if img_crop is None:
                msg = 'Can not recognise the detection box in ' + self.filePath + '. Please change manually'
                QMessageBox.information(self, "Information", msg)
                return

            # merge the text result in the cell
            texts = ''
            probs = 0. # the probability of the cell is avgerage prob of every text box in the cell
            bboxes = self.ocr.ocr(img_crop, det=True, rec=False, cls=False)[0]
            if len(bboxes) > 0:
                bboxes.reverse() # top row text at first
                for _bbox in bboxes:
                    patch = get_rotate_crop_image(img_crop, np.array(_bbox, np.float32))
                    rec_res = self.ocr.ocr(patch, det=False, rec=True, cls=False)[0]
                    text = rec_res[0][0]
                    if text != '':
                        texts += text + ('' if text[0].isalpha() else ' ') # add space between english word
                        probs += rec_res[0][1]
                probs = probs / len(bboxes)
            result = [(texts.strip(), probs)]

            if result[0][0] != '':
                result.insert(0, box)
                print('result in reRec is ', result)
                if result[1][0] == shape.label:
                    print('label no change')
                else:
                    shape.label = result[1][0]
            else:
                print('Can not recognise the box')
                if self.noLabelText == shape.label:
                    print('label no change')
                else:
                    shape.label = self.noLabelText
            self.singleLabel(shape)
            self.setDirty()

    def exportJSON(self):
        '''
            export PPLabel and CSV to JSON (PubTabNet)
        '''
        import pandas as pd

        # automatically save annotations
        self.saveFilestate()
        self.savePPlabel(mode='auto')

        # load box annotations
        labeldict = {}
        if not os.path.exists(self.PPlabelpath):
            msg = 'ERROR, Can not find Label.txt'
            QMessageBox.information(self, "Information", msg)
            return
        else:
            with open(self.PPlabelpath, 'r', encoding='utf-8') as f:
                data = f.readlines()
                for each in data:
                    file, label = each.split('\t')
                    if label:
                        label = label.replace('false', 'False')
                        label = label.replace('true', 'True')
                        labeldict[file] = eval(label)
                    else:
                        labeldict[file] = []
        
        # read table recognition output
        TableRec_excel_dir = os.path.join(
            self.lastOpenDir, 'tableRec_excel_output')

        # save txt
        fid = open(
            "{}/gt.txt".format(self.lastOpenDir), "w", encoding='utf-8')
        for image_path in labeldict.keys():
            # load csv annotations
            filename, _ = os.path.splitext(os.path.basename(image_path))
            csv_path = os.path.join(
                TableRec_excel_dir, filename + '.xlsx')
            if not os.path.exists(csv_path):
                continue

            excel = xlrd.open_workbook(csv_path)
            sheet0 = excel.sheet_by_index(0)  # only sheet 0
            merged_cells = sheet0.merged_cells # (0,1,1,3) start row, end row, start col, end col

            html_list = [['td'] * sheet0.ncols for i in range(sheet0.nrows)]

            for merged in merged_cells:
                html_list = expand_list(merged, html_list)

            token_list = convert_token(html_list)

            # load box annotations
            cells = []
            for anno in labeldict[image_path]:
                tokens = list(anno['transcription'])
                cells.append({
                    'tokens': tokens, 
                    'bbox': anno['points']
                    })

            # 构造标注信息
            html = {
                'structure': {
                    'tokens': token_list
                    }, 
                'cells': cells
                }
            d = {
                'filename': os.path.basename(image_path), 
                'html': html
                }
            # 重构HTML
            d['gt'] = rebuild_html_from_ppstructure_label(d)
            fid.write('{}\n'.format(
                json.dumps(
                    d, ensure_ascii=False)))
                    
        # convert to PP-Structure label format
        fid.close()
        msg = 'JSON sucessfully saved in {}/gt.txt'.format(self.lastOpenDir)
        QMessageBox.information(self, "Information", msg)

    def autolcm(self):
        vbox = QVBoxLayout()
        hbox = QHBoxLayout()
        self.panel = QLabel()
        self.panel.setText(self.stringBundle.getString('choseModelLg'))
        self.panel.setAlignment(Qt.AlignLeft)
        self.comboBox = QComboBox()
        self.comboBox.setObjectName("comboBox")
        self.comboBox.addItems(['Chinese & English', 'English', 'French', 'German', 'Korean', 'Japanese'])
        vbox.addWidget(self.panel)
        vbox.addWidget(self.comboBox)
        self.dialog = QDialog()
        self.dialog.resize(300, 100)
        self.okBtn = QPushButton(self.stringBundle.getString('ok'))
        self.cancelBtn = QPushButton(self.stringBundle.getString('cancel'))

        self.okBtn.clicked.connect(self.modelChoose)
        self.cancelBtn.clicked.connect(self.cancel)
        self.dialog.setWindowTitle(self.stringBundle.getString('choseModelLg'))

        hbox.addWidget(self.okBtn)
        hbox.addWidget(self.cancelBtn)

        vbox.addWidget(self.panel)
        vbox.addLayout(hbox)
        self.dialog.setLayout(vbox)
        self.dialog.setWindowModality(Qt.ApplicationModal)
        self.dialog.exec_()
        if self.filePath:
            self.AutoRecognition.setEnabled(True)
            self.actions.AutoRec.setEnabled(True)

    def modelChoose(self):
        print(self.comboBox.currentText())
        lg_idx = {'Chinese & English': 'ch', 'English': 'en', 'French': 'french', 'German': 'german',
                  'Korean': 'korean', 'Japanese': 'japan'}
        del self.ocr
        self.ocr = PaddleOCR(use_pdserving=False, use_angle_cls=True, det=True, cls=True, use_gpu=False,
                             lang=lg_idx[self.comboBox.currentText()])
        del self.table_ocr
        self.table_ocr = PPStructure(use_pdserving=False,
                                     use_gpu=False,
                                     lang=lg_idx[self.comboBox.currentText()],
                                     layout=False,
                                     show_log=False)
        self.dialog.close()

    def cancel(self):
        self.dialog.close()

    def loadFilestate(self, saveDir):
        self.fileStatepath = saveDir + '/fileState.txt'
        self.fileStatedict = {}
        if not os.path.exists(self.fileStatepath):
            f = open(self.fileStatepath, 'w', encoding='utf-8')
        else:
            with open(self.fileStatepath, 'r', encoding='utf-8') as f:
                states = f.readlines()
                for each in states:
                    file, state = each.split('\t')
                    self.fileStatedict[file] = 1
                self.actions.saveLabel.setEnabled(True)
                self.actions.saveRec.setEnabled(True)
                self.actions.exportJSON.setEnabled(True)

    def saveFilestate(self):
        with open(self.fileStatepath, 'w', encoding='utf-8') as f:
            for key in self.fileStatedict:
                f.write(key + '\t')
                f.write(str(self.fileStatedict[key]) + '\n')

    def loadLabelFile(self, labelpath):
        labeldict = {}
        if not os.path.exists(labelpath):
            f = open(labelpath, 'w', encoding='utf-8')

        else:
            with open(labelpath, 'r', encoding='utf-8') as f:
                data = f.readlines()
                for each in data:
                    file, label = each.split('\t')
                    if label:
                        label = label.replace('false', 'False')
                        label = label.replace('true', 'True')
                        labeldict[file] = eval(label)
                    else:
                        labeldict[file] = []
        return labeldict

    def savePPlabel(self, mode='Manual'):
        savedfile = [self.getImglabelidx(i) for i in self.fileStatedict.keys()]
        with open(self.PPlabelpath, 'w', encoding='utf-8') as f:
            for key in self.PPlabel:
                if key in savedfile and self.PPlabel[key] != []:
                    f.write(key + '\t')
                    f.write(json.dumps(self.PPlabel[key], ensure_ascii=False) + '\n')

        if mode == 'Manual':
            if self.lang == 'ch':
                msg = '已将检查过的图片标签保存在 ' + self.PPlabelpath + " 文件中"
            else:
                msg = 'Images that have been checked are saved in ' + self.PPlabelpath
            QMessageBox.information(self, "Information", msg)

    def saveCacheLabel(self):
        with open(self.Cachelabelpath, 'w', encoding='utf-8') as f:
            for key in self.Cachelabel:
                f.write(key + '\t')
                f.write(json.dumps(self.Cachelabel[key], ensure_ascii=False) + '\n')

    def saveLabelFile(self):
        self.saveFilestate()
        self.savePPlabel()

    def saveRecResult(self):
        if {} in [self.PPlabelpath, self.PPlabel, self.fileStatedict]:
            QMessageBox.information(self, "Information", "Check the image first")
            return

        rec_gt_dir = os.path.dirname(self.PPlabelpath) + '/rec_gt.txt'
        crop_img_dir = os.path.dirname(self.PPlabelpath) + '/crop_img/'
        ques_img = []
        if not os.path.exists(crop_img_dir):
            os.mkdir(crop_img_dir)

        with open(rec_gt_dir, 'w', encoding='utf-8') as f:
            for key in self.fileStatedict:
                idx = self.getImglabelidx(key)
                try:
                    img = cv2.imread(key)
                    for i, label in enumerate(self.PPlabel[idx]):
                        if label['difficult']:
                            continue
                        img_crop = get_rotate_crop_image(img, np.array(label['points'], np.float32))
                        img_name = os.path.splitext(os.path.basename(idx))[0] + '_crop_' + str(i) + '.jpg'
                        cv2.imwrite(crop_img_dir + img_name, img_crop)
                        f.write('crop_img/' + img_name + '\t')
                        f.write(label['transcription'] + '\n')
                except Exception as e:
                    ques_img.append(key)
                    print("Can not read image ", e)
        if ques_img:
            QMessageBox.information(self,
                                    "Information",
                                    "The following images can not be saved, please check the image path and labels.\n"
                                    + "".join(str(i) + '\n' for i in ques_img))
        QMessageBox.information(self, "Information", "Cropped images have been saved in " + str(crop_img_dir))

    def speedChoose(self):
        if self.labelDialogOption.isChecked():
            self.canvas.newShape.disconnect()
            self.canvas.newShape.connect(partial(self.newShape, True))

        else:
            self.canvas.newShape.disconnect()
            self.canvas.newShape.connect(partial(self.newShape, False))

    def autoSaveFunc(self):
        if self.autoSaveOption.isChecked():
            self.autoSaveNum = 1  # Real auto_Save
            try:
                self.saveLabelFile()
            except:
                pass
            print('The program will automatically save once after confirming an image')
        else:
            self.autoSaveNum = 5  # Used for backup
            print('The program will automatically save once after confirming 5 images (default)')

    def change_box_key(self):
        if not self.kie_mode:
            return
        key_text, _ = self.keyDialog.popUp(self.key_previous_text)
        if key_text is None:
            return
        self.key_previous_text = key_text
        for shape in self.canvas.selectedShapes:
            shape.key_cls = key_text
            if not self.keyList.findItemsByLabel(key_text):
                item = self.keyList.createItemFromLabel(key_text)
                self.keyList.addItem(item)
                rgb = self._get_rgb_by_label(key_text, self.kie_mode)
                self.keyList.setItemLabel(item, key_text, rgb)

            self._update_shape_color(shape)
            self.keyDialog.addLabelHistory(key_text)
            
        # save changed shape
        self.setDirty()

    def undoShapeEdit(self):
        self.canvas.restoreShape()
        self.labelList.clear()
        self.indexList.clear()
        self.BoxList.clear()
        self.loadShapes(self.canvas.shapes)
        self.actions.undo.setEnabled(self.canvas.isShapeRestorable)

    def loadShapes(self, shapes, replace=True):
        self._noSelectionSlot = True
        for shape in shapes:
            self.addLabel(shape)
        self.labelList.clearSelection()
        self.indexList.clearSelection()
        self._noSelectionSlot = False
        self.canvas.loadShapes(shapes, replace=replace)
        print("loadShapes")  # 1

    def lockSelectedShape(self):
        """lock the selected shapes.

        Add self.selectedShapes to lock self.canvas.lockedShapes, 
        which holds the ratio of the four coordinates of the locked shapes
        to the width and height of the image
        """
        width, height = self.image.width(), self.image.height()

        def format_shape(s):
            return dict(label=s.label,  # str
                        line_color=s.line_color.getRgb(),
                        fill_color=s.fill_color.getRgb(),
                        ratio=[[int(p.x()) / width, int(p.y()) / height] for p in s.points],  # QPonitF
                        difficult=s.difficult,  # bool
                        key_cls=s.key_cls,  # bool
                        )

        # lock
        if len(self.canvas.lockedShapes) == 0:
            for s in self.canvas.selectedShapes:
                s.line_color = DEFAULT_LOCK_COLOR
                s.locked = True
            shapes = [format_shape(shape) for shape in self.canvas.selectedShapes]
            trans_dic = []
            for box in shapes:
                trans_dict = {"transcription": box['label'], "ratio": box['ratio'], "difficult": box['difficult']}
                if self.kie_mode:
                    trans_dict.update({"key_cls": box["key_cls"]})
                trans_dic.append(trans_dict)
            self.canvas.lockedShapes = trans_dic
            self.actions.save.setEnabled(True)

        # unlock
        else:
            for s in self.canvas.shapes:
                s.line_color = DEFAULT_LINE_COLOR
            self.canvas.lockedShapes = []
            self.result_dic_locked = []
            self.setDirty()
            self.actions.save.setEnabled(True)


def inverted(color):
    return QColor(*[255 - v for v in color.getRgb()])


def read(filename, default=None):
    try:
        with open(filename, 'rb') as f:
            return f.read()
    except:
        return default


def str2bool(v):
    return v.lower() in ("true", "t", "1")


def get_main_app(argv=[]):
    """
    Standard boilerplate Qt application code.
    Do everything but app.exec_() -- so that we can test the application in one thread
    """
    app = QApplication(argv)
    app.setApplicationName(__appname__)
    app.setWindowIcon(newIcon("app"))
    # Tzutalin 201705+: Accept extra arguments to change predefined class file
    arg_parser = argparse.ArgumentParser()
    arg_parser.add_argument("--lang", type=str, default='en', nargs="?")
    arg_parser.add_argument("--gpu", type=str2bool, default=True, nargs="?")
    arg_parser.add_argument("--kie", type=str2bool, default=False, nargs="?")
    arg_parser.add_argument("--predefined_classes_file",
                            default=os.path.join(os.path.dirname(__file__), "data", "predefined_classes.txt"),
                            nargs="?")
    args = arg_parser.parse_args(argv[1:])

    win = MainWindow(lang=args.lang,
                     gpu=args.gpu,
                     kie_mode=args.kie,
                     default_predefined_class_file=args.predefined_classes_file)
    win.show()
    return app, win


def main():
    """construct main app and run it"""
    app, _win = get_main_app(sys.argv)
    return app.exec_()


if __name__ == '__main__':

    resource_file = './libs/resources.py'
    if not os.path.exists(resource_file):
        output = os.system('pyrcc5 -o libs/resources.py resources.qrc')
        assert output == 0, "operate the cmd have some problems ,please check  whether there is a in the lib " \
                            "directory resources.py "

    sys.exit(main())
